In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [2]:
# import sys
# !{sys.executable} -m pip install optuna

In [3]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

Using TensorFlow backend.


In [4]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import json
import copy
import gc
import warnings
from tqdm import tqdm_notebook, tqdm

import optuna

import lightgbm as lgb
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

import eli5
from eli5.sklearn import PermutationImportance

import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
csv_file_folder =  '../../data/input'
os.listdir(csv_file_folder)

['structures',
 'magnetic_shielding_parameters.csv',
 'train.csv',
 'dipole_moments.csv',
 'magnetic_shielding_tensors.csv',
 'mulliken_charges.csv',
 'potential_energy.csv',
 'sample_submission.csv',
 'scalar_coupling_contributions.csv',
 'structures.csv',
 'test.csv']

In [6]:
file_folder =  '../../data/feature'
[f for f in os.listdir(file_folder) if  (not f.startswith('.'))]

['angles-and-distances_test.pkl',
 'angles-and-distances_train.pkl',
 'brute-force-feature-engineering_test.pkl',
 'eem_test.pkl',
 'giba-r-data-table-simple-features-0-991-lb_test.pkl',
 'giba-r-data-table-simple-features-0-991-lb_train.pkl',
 'brute-force-feature-engineering_train.pkl',
 'keras-neural-net-for-champs_test.pkl',
 'keras-neural-net-for-champs_train.pkl',
 'eem_train.pkl',
 'molecular-properties-eda-and-models_test.pkl',
 'molecular-properties-eda-and-models_train.pkl',
 'molecule-with-openbabel_test.pkl',
 'molecule-with-openbabel_train.pkl',
 'dataset-with-number-of-bonds-between-atoms_test.pkl',
 'brute-force-feature-engineering-mini_test.pkl',
 'df_train.gzde',
 'df_test.gzde',
 'eachtype_train.pkl',
 'eachtype_test.pkl',
 'giba-r-data-table-simple-features-1-17-lb_test.pkl',
 'dataset-with-number-of-bonds-between-atoms_train.pkl',
 'brute-force-feature-engineering-mini_train.pkl',
 'pytorch_geometric_test.pkl',
 'pytorch_geometric_train.pkl',
 'giba-r-data-table-sim

In [6]:
scalar_coupling_constant_columns_list = [{'columns': ['cos_center1', 'adC3', 'dist_H_4_y', 'inv_dist1R', 'eem_1', 'dist_C_0_y', 'dist_to_type_std', 'cos_f0', 'molecule_dist_min', 'inv_dist1', 'eem_0', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_std_diff', 'tertiary_angle_0', 'tertiary_distance_5', 'dist_O_1_x', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_4_y', 'dist_to_type_mean', 'inv_distPR', 'inv_dist1E', 'adC1', 'dist_C_0_x', 'dist_H_1_x', 'dist_O_0_x', 'adN1', 'dist_to_type_1_mean', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'dist_H_3_x', 'tertiary_angle_1', 'linkM0', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'molecule_type_dist_mean_diff', 'dist_to_type_0_mean', 'dist_C_3_x', 'tertiary_distance_4', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'dist_H_0_y', 'tertiary_distance_0', 'dist_C_2_x', 'tertiary_angle_5', 'dist_H_0_x', 'coulomb_H.y', 'tertiary_distance_1', 'dist_H_2_y', 'dist_O_1_y', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dist_O_0_y', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'distC1', 'dist_xyz', 'dist_C_2_y', 'tertiary_angle_2', 'tertiary_atom_0', 'dist_H_3_y', 'tertiary_atom_1', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'dist_C_1_y', 'atom_1_bond_lengths_max', 'distC0', 'vander_H.x', 'dist_H_2_x', 'adC2', 'yukawa_H.y', 'molecule_atom_index_1_dist_min_diff', 'cos_center0', 'dist_C_4_x', 'dist_N_0_x', 'atom_index_farthest_0', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['min_molecule_atom_1_dist_xyz', 'tertiary_distance_1', 'adC3', 'dist_H_2_y', 'inv_dist1R', 'dist_C_0_y', 'cos_f0', 'atom_1_bond_lengths_std', 'inv_dist1', 'eem_0', 'dist_O_0_y', 'molecule_atom_index_0_dist_max_div', 'cos_center0_center1', 'tertiary_angle_0', 'tertiary_distance_5', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'dist_C_2_y', 'distance_center0', 'distN0', 'vander_N.x', 'inv_distPR', 'tertiary_angle_2', 'tertiary_atom_0', 'dist_H_3_y', 'adC4', 'dist_C_0_x', 'dist_H_1_x', 'tertiary_angle_3', 'dist_O_0_x', 'dist_C_1_y', 'atom_1_bond_lengths_max', 'molecule_atom_index_1_dist_max_div', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'tertiary_angle_1', 'dist_C_1_x', 'vander_H.x', 'dist_to_type_0_mean', 'inv_distP', 'dist_C_3_x', 'tertiary_distance_4', 'bond_atom', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'vander_H.y', 'dist_N_1_x', 'dist_H_0_y', 'yukawa_H.y', 'tertiary_distance_0', 'yukawa_H.x', 'dist_C_2_x', 'linkN', 'dist_N_0_x', 'dist_H_0_x', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_6', 'adC3', 'inv_dist1R', 'eem_1', 'cos_f0', 'dist_C_0_y', 'cos_c1', 'yukawa_O.x', 'inv_dist1', 'tertiary_atom_2', 'molecule_atom_index_0_dist_max_div', 'tertiary_distance_2', 'tertiary_angle_0', 'dist_O_1_x', 'tertiary_distance_3', 'mean_molecule_atom_0_dist_xyz', 'inv_distPR', 'min_molecule_atom_0_dist_xyz', 'molecule_type_dist_max', 'dist_C_0_x', 'dist_H_1_x', 'adC1', 'dist_O_0_x', 'molecule_atom_index_0_dist_max_diff', 'adN1', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'tertiary_angle_1', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'inv_dist0', 'dist_C_3_x', 'atom_0_bond_lengths_max', 'atom_1_bond_lengths_min', 'bond_atom', 'inv_dist0R', 'dist_H_0_y', 'molecule_atom_index_1_dist_min_div', 'dist_C_2_x', 'cos_c0', 'dist_H_0_x', 'tertiary_distance_1', 'dist_H_2_y', 'tertiary_atom_3', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dist_O_0_y', 'vander_O.y', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'dist_C_2_y', 'yukawa_O.y', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_2', 'tertiary_atom_0', 'tertiary_atom_1', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'atom_1_bond_lengths_max', 'dist_C_1_y', 'distC0', 'adC2', 'tertiary_atom_4', 'cos_c0_c1', 'atom_index_1_n_cycle', 'dist_N_0_x', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['max_molecule_atom_0_dist_xyz', 'tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_1_dist_min_diff', 'dist_C_4_x', 'adC3', 'distN0', 'atom_0_bond_lengths_max', 'dist_C_3_y', 'tertiary_angle_1', 'atom_1_bond_lengths_mean', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'molecule_atom_index_1_dist_max_div', 'molecule_atom_index_0_dist_min_div', 'dist_C_0_x', 'adC1', 'tertiary_distance_1', 'atom_1_bond_lengths_std', 'dist_H_0_y', 'dist_xyz', 'dist_O_0_x', 'inv_distPR', 'cos_c0_c1', 'cos_c0', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_C_3_x', 'atom_1_n_bonds', 'bond_atom', 'dist_C_2_x', 'dist_N_0_y', 'dist_N_0_x', 'atom_index_1_cycle_size_mean', 'min_molecule_atom_0_dist_xyz', 'tertiary_angle_3', 'tertiary_distance_3', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}]

In [7]:
pso_columns_list = [{'columns': ['dist_to_type_0_mean', 'atom_1_n_bonds', 'distance_center0', 'dist_to_type_std', 'dist_to_type_1_mean', 'dist_H_3_y', 'tertiary_distance_1', 'tertiary_atom_1', 'adC1', 'distC1', 'dist_O_0_x', 'yukawa_C.x', 'dist_C_3_y', 'inv_distPR', 'eem_0', 'dist_H_1_y', 'dist_H_4_y', 'dist_C_4_y', 'tertiary_angle_0', 'distance_farthest_0', 'tertiary_angle_2', 'distance_center1', 'tertiary_angle_3', 'tertiary_atom_0', 'dist_O_1_y', 'tertiary_atom_2', 'vander_C.x', 'coulomb_H.y', 'dist_H_2_x', 'dist_C_4_x', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_H_3_x', 'dist_O_0_y', 'dist_H_2_y', 'dist_N_1_x', 'bond_atom', 'inv_dist0R', 'atom_1_bond_lengths_mean', 'inv_dist1', 'dist_C_2_y', 'adN1', 'molecule_dist_min', 'dist_H_0_y', 'atom_index_1_cycle_size_mean', 'dist_C_3_x', 'eem_1', 'dist_C_1_x', 'dist_C_0_y', 'dist_H_0_x', 'tertiary_distance_0', 'tertiary_distance_2', 'vander_H.y', 'tertiary_angle_1', 'adC2', 'dist_N_0_y', 'dist_C_0_x', 'dist_H_1_x', 'dist_H_4_x', 'molecule_name.1', 'atom_1_bond_lengths_std', 'adC3', 'tertiary_distance_5', 'yukawa_N.x', 'link0', 'dist_to_type_mean', 'molecule_atom_index_0_dist_max_div', 'dist_N_0_x', 'tertiary_atom_3', 'tertiary_distance_9', 'dist_C_1_y', 'tertiary_distance_3', 'inv_dist1R', 'molecule_atom_index_1_dist_min_diff', 'vander_O.y', 'tertiary_distance_4', 'molecule_atom_index_0_dist_mean_div', 'dist_C_2_x', 'dist_O_1_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_to_type_0_mean', 'atom_1_n_bonds', 'distance_center0', 'dist_to_type_std', 'dist_to_type_1_mean', 'dist_H_3_y', 'tertiary_distance_1', 'tertiary_atom_1', 'adC1', 'distC1', 'dist_O_0_x', 'yukawa_C.x', 'dist_C_3_y', 'inv_distPR', 'eem_0', 'dist_H_1_y', 'dist_H_4_y', 'dist_C_4_y', 'tertiary_angle_0', 'distance_farthest_0', 'tertiary_angle_2', 'distance_center1', 'tertiary_angle_3', 'tertiary_atom_0', 'dist_O_1_y', 'tertiary_atom_2', 'vander_C.x', 'coulomb_H.y', 'dist_H_2_x', 'dist_C_4_x', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_H_3_x', 'dist_O_0_y', 'dist_H_2_y', 'dist_N_1_x', 'bond_atom', 'inv_dist0R', 'atom_1_bond_lengths_mean', 'inv_dist1', 'dist_C_2_y', 'adN1', 'molecule_dist_min', 'dist_H_0_y', 'atom_index_1_cycle_size_mean', 'dist_C_3_x', 'eem_1', 'dist_C_1_x', 'dist_C_0_y', 'dist_H_0_x', 'tertiary_distance_0', 'tertiary_distance_2', 'vander_H.y', 'tertiary_angle_1', 'adC2', 'dist_N_0_y', 'dist_C_0_x', 'dist_H_1_x', 'dist_H_4_x', 'molecule_name.1', 'atom_1_bond_lengths_std', 'adC3', 'tertiary_distance_5', 'yukawa_N.x', 'link0', 'dist_to_type_mean', 'molecule_atom_index_0_dist_max_div', 'dist_N_0_x', 'tertiary_atom_3', 'tertiary_distance_9', 'dist_C_1_y', 'tertiary_distance_3', 'inv_dist1R', 'molecule_atom_index_1_dist_min_diff', 'vander_O.y', 'tertiary_distance_4', 'molecule_atom_index_0_dist_mean_div', 'dist_C_2_x', 'dist_O_1_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_to_type_0_mean', 'atom_1_n_bonds', 'distance_center0', 'dist_to_type_std', 'dist_to_type_1_mean', 'dist_H_3_y', 'tertiary_distance_1', 'tertiary_atom_1', 'adC1', 'distC1', 'dist_O_0_x', 'yukawa_C.x', 'dist_C_3_y', 'inv_distPR', 'eem_0', 'dist_H_1_y', 'dist_H_4_y', 'dist_C_4_y', 'tertiary_angle_0', 'distance_farthest_0', 'tertiary_angle_2', 'distance_center1', 'tertiary_angle_3', 'tertiary_atom_0', 'dist_O_1_y', 'tertiary_atom_2', 'vander_C.x', 'coulomb_H.y', 'dist_H_2_x', 'dist_C_4_x', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_H_3_x', 'dist_O_0_y', 'dist_H_2_y', 'dist_N_1_x', 'bond_atom', 'inv_dist0R', 'atom_1_bond_lengths_mean', 'inv_dist1', 'dist_C_2_y', 'adN1', 'molecule_dist_min', 'dist_H_0_y', 'atom_index_1_cycle_size_mean', 'dist_C_3_x', 'eem_1', 'dist_C_1_x', 'dist_C_0_y', 'dist_H_0_x', 'tertiary_distance_0', 'tertiary_distance_2', 'vander_H.y', 'tertiary_angle_1', 'adC2', 'dist_N_0_y', 'dist_C_0_x', 'dist_H_1_x', 'dist_H_4_x', 'molecule_name.1', 'atom_1_bond_lengths_std', 'adC3', 'tertiary_distance_5', 'yukawa_N.x', 'link0', 'dist_to_type_mean', 'molecule_atom_index_0_dist_max_div', 'dist_N_0_x', 'tertiary_atom_3', 'tertiary_distance_9', 'dist_C_1_y', 'tertiary_distance_3', 'inv_dist1R', 'molecule_atom_index_1_dist_min_diff', 'vander_O.y', 'tertiary_distance_4', 'molecule_atom_index_0_dist_mean_div', 'dist_C_2_x', 'dist_O_1_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_to_type_0_mean', 'tertiary_distance_16', 'tertiary_distance_15', 'dist_to_type_1_mean', 'tertiary_distance_14', 'yukawa_O.x', 'tertiary_atom_1', 'cos_c0_c1', 'dist_O_0_x', 'yukawa_C.x', 'dist_C_3_y', 'yukawa_N.y', 'dist_H_1_y', 'inv_distPR', 'tertiary_distance_11', 'coulomb_C.y', 'dist_C_4_y', 'tertiary_angle_0', 'tertiary_angle_3', 'tertiary_atom_0', 'cos_c0', 'yukawa_C.y', 'dist_xyz', 'dist_O_1_y', 'tertiary_atom_2', 'vander_C.y', 'vander_C.x', 'coulomb_H.y', 'dist_H_2_x', 'dist_C_4_x', 'tertiary_distance_10', 'dist_H_3_x', 'tertiary_distance_7', 'vander_N.y', 'tertiary_atom_19', 'dist_O_0_y', 'dist_N_1_x', 'tertiary_atom_17', 'yukawa_O.y', 'inv_dist0R', 'tertiary_distance_19', 'atom_1_bond_lengths_mean', 'dist_C_2_y', 'adN1', 'inv_distPE', 'molecule_atom_index_0_dist_min_div', 'dist_F_0_x', 'molecule_dist_min', 'yukawa_F.y', 'tertiary_distance_6', 'dist_C_3_x', 'molecule_dist_mean', 'tertiary_distance_12', 'cos_c1', 'dist_C_1_x', 'dist_C_0_y', 'tertiary_angle_1', 'adC2', 'dist_N_0_y', 'dist_C_0_x', 'dist_H_1_x', 'dist_H_4_x', 'cos_f0_f1', 'adC3', 'dist_N_1_y', 'yukawa_N.x', 'tertiary_distance_8', 'link0', 'dist_to_type_mean', 'dist_N_0_x', 'tertiary_distance_9', 'molecule_atom_index_0_dist_min_diff', 'atom_0_bond_lengths_max', 'dist_C_1_y', 'vander_O.x', 'tertiary_distance_3', 'tertiary_distance_13', 'tertiary_distance_4', 'dist_C_2_x', 'dist_O_1_x', 'atom_n'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_atom_0', 'atom_1_bond_lengths_max', 'atom_1_n_bonds', 'tertiary_atom_1', 'dist_C_0_y', 'molecule_atom_index_0_dist_max_diff', 'cos_c0', 'tertiary_angle_2', 'tertiary_distance_2', 'molecule_atom_index_0_dist_min_div', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_1', 'atom_0_bond_lengths_max', 'eem_1', 'tertiary_distance_1', 'dist_C_1_x', 'atom_index_1_cycle_size_mean', 'cos_f0', 'adC2', 'inv_dist1R', 'tertiary_atom_2', 'molecule_atom_index_1_dist_min_div', 'atom_1_bond_lengths_mean', 'inv_distPR', 'dist_to_type_1_mean', 'dist_N_0_x', 'dist_to_type_0_mean', 'dist_to_type_mean', 'dist_xyz', 'dist_H_1_y', 'dist_H_2_y', 'dist_O_0_y', 'atom_1_bond_lengths_min', 'dist_C_1_y', 'adC3', 'dist_N_0_y', 'dist_C_2_y', 'molecule_atom_index_0_dist_max_div', 'eem_0', 'dist_C_0_x', 'inv_dist0R', 'dist_C_3_y', 'cos_c0_c1', 'tertiary_distance_3', 'adN1', 'sd_molecule_atom_0_dist_xyz', 'inv_dist0E', 'dist_C_3_x', 'min_molecule_atom_1_dist_xyz', 'tertiary_angle_0', 'vander_C.y', 'distance_farthest_0', 'molecule_atom_index_0_dist_mean_div', 'molecule_type_dist_max', 'atom_1_bond_lengths_std', 'tertiary_angle_3', 'min_molecule_atom_0_dist_xyz', 'dist_C_2_x', 'dist_H_0_x', 'dist_O_0_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_atom_0', 'atom_1_bond_lengths_max', 'atom_1_n_bonds', 'tertiary_atom_1', 'dist_C_0_y', 'molecule_atom_index_0_dist_max_diff', 'cos_c0', 'tertiary_angle_2', 'tertiary_distance_2', 'molecule_atom_index_0_dist_min_div', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_1', 'atom_0_bond_lengths_max', 'eem_1', 'tertiary_distance_1', 'dist_C_1_x', 'atom_index_1_cycle_size_mean', 'cos_f0', 'adC2', 'inv_dist1R', 'tertiary_atom_2', 'molecule_atom_index_1_dist_min_div', 'atom_1_bond_lengths_mean', 'inv_distPR', 'dist_to_type_1_mean', 'dist_N_0_x', 'dist_to_type_0_mean', 'dist_to_type_mean', 'dist_xyz', 'dist_H_1_y', 'dist_H_2_y', 'dist_O_0_y', 'atom_1_bond_lengths_min', 'dist_C_1_y', 'adC3', 'dist_N_0_y', 'dist_C_2_y', 'molecule_atom_index_0_dist_max_div', 'eem_0', 'dist_C_0_x', 'inv_dist0R', 'dist_C_3_y', 'cos_c0_c1', 'tertiary_distance_3', 'adN1', 'sd_molecule_atom_0_dist_xyz', 'inv_dist0E', 'dist_C_3_x', 'min_molecule_atom_1_dist_xyz', 'tertiary_angle_0', 'vander_C.y', 'distance_farthest_0', 'molecule_atom_index_0_dist_mean_div', 'molecule_type_dist_max', 'atom_1_bond_lengths_std', 'tertiary_angle_3', 'min_molecule_atom_0_dist_xyz', 'dist_C_2_x', 'dist_H_0_x', 'dist_O_0_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['adC1', 'yukawa_C.x', 'dist_C_2_y', 'yukawa_F.y', 'molecule_atom_1_dist_min_div', 'molecule_couples', 'dist_to_type_mean', 'molecule_atom_index_0_dist_min_diff', 'dist_C_3_y', 'tertiary_distance_12', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_11', 'min_molecule_atom_0_dist_xyz', 'dist_C_0_x', 'cos_c0', 'dist_O_1_x', 'yukawa_O.x', 'dist_N_1_x', 'yukawa_N.y', 'tertiary_distance_16', 'dist_C_0_y', 'vander_C.y', 'dist_to_type_std', 'tertiary_atom_16', 'yukawa_O.y', 'dist_O_0_x', 'yukawa_C.y', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_distance_13', 'link0', 'dist_N_0_y', 'dist_O_0_y', 'inv_dist0R', 'tertiary_angle_2', 'vander_C.x', 'tertiary_angle_1', 'dist_C_4_x', 'dist_O_1_y', 'dist_xyz', 'tertiary_distance_6', 'dist_N_1_y', 'dist_N_0_x', 'molecule_name.1', 'dist_C_2_x', 'molecule_atom_index_1_dist_std_diff', 'distance_f1', 'cos_c0_c1', 'cos_c1', 'tertiary_distance_10', 'tertiary_distance_9', 'tertiary_atom_15', 'dist_C_1_x', 'molecule_atom_1_dist_min_diff', 'yukawa_H.y', 'tertiary_atom_17', 'tertiary_angle_0', 'inv_distPR', 'distance_c1', 'coulomb_H.y', 'dist_to_type_1_mean', 'atom_1_bond_lengths_mean', 'tertiary_distance_7', 'dist_H_0_x', 'dist_H_4_x', 'tertiary_distance_3', 'tertiary_distance_14', 'max_molecule_atom_0_dist_xyz', 'dist_to_type_0_mean', 'dist_O_2_y', 'tertiary_distance_8', 'dist_C_1_y', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_1', 'tertiary_angle_3', 'dist_C_3_x', 'molecule_dist_mean', 'dist_C_4_y', 'dist_H_0_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['adC1', 'dist_C_2_y', 'dist_H_1_y', 'tertiary_distance_5', 'tertiary_angle_4', 'tertiary_distance_2', 'dist_to_type_mean', 'tertiary_distance_4', 'molecule_atom_index_0_dist_min_diff', 'dist_C_3_y', 'min_molecule_atom_0_dist_xyz', 'dist_C_0_x', 'vander_N.y', 'cos_c0', 'dist_H_1_x', 'eem_1', 'dist_N_1_x', 'atom_0_bond_lengths_max', 'dist_C_0_y', 'dist_to_type_std', 'tertiary_atom_3', 'atom_1_bond_lengths_max', 'inv_distPE', 'distance_farthest_0', 'dist_O_0_x', 'molecule_atom_index_0_dist_max_div', 'adC3', 'yukawa_C.y', 'adC2', 'molecule_atom_index_0_dist_mean_diff', 'dist_N_0_y', 'dist_O_0_y', 'vander_O.y', 'cos_f0', 'inv_dist0R', 'inv_dist1R', 'tertiary_angle_2', 'atom_1_bond_lengths_std', 'yukawa_N.x', 'tertiary_angle_1', 'dist_C_4_x', 'dist_xyz', 'tertiary_distance_6', 'dist_H_2_y', 'dist_N_1_y', 'dist_N_0_x', 'molecule_atom_index_0_dist_max_diff', 'dist_C_2_x', 'molecule_atom_index_1_dist_std_diff', 'cos_c0_c1', 'cos_c1', 'inv_dist1', 'tertiary_distance_10', 'dist_C_1_x', 'tertiary_angle_0', 'inv_distPR', 'dist_to_type_1_mean', 'atom_1_bond_lengths_mean', 'atom_1_n_bonds', 'min_molecule_atom_1_dist_xyz', 'dist_H_0_x', 'tertiary_distance_3', 'tertiary_atom_1', 'atom_index_1_cycle_size_mean', 'dist_to_type_0_mean', 'coulomb_H.x', 'mean_molecule_atom_0_dist_xyz', 'dist_C_1_y', 'distN0', 'molecule_atom_index_1_dist_min_diff', 'molecule_atom_index_0_dist_min_div', 'vander_N.x', 'tertiary_angle_3', 'tertiary_distance_1', 'dist_C_3_x', 'tertiary_atom_0', 'dist_H_0_y', 'molecule_type_0_dist_std_diff'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}]

In [8]:
sd_columns_list = [{'columns': ['tertiary_angle_4', 'adN1', 'vander_H.y', 'tertiary_distance_0', 'tertiary_angle_5', 'dist_C_2_y', 'dist_C_4_y', 'atom_1_bond_lengths_std', 'tertiary_angle_0', 'dist_H_2_x', 'dist_O_1_y', 'tertiary_distance_8', 'molecule_dist_min', 'atom_index_1_cycle_size_mean', 'molecule_atom_index_1_dist_min_diff', 'adC2', 'dist_H_4_y', 'distC1', 'dist_to_type_0_mean', 'dist_C_2_x', 'dist_to_type_mean', 'atom_1_bond_lengths_mean', 'inv_dist1', 'dist_O_0_y', 'distance_farthest_0', 'molecule_atom_index_0_dist_max_div', 'dist_H_2_y', 'inv_distPR', 'vander_O.x', 'tertiary_angle_3', 'cos_f0', 'adC1', 'molecule_type_dist_std_diff', 'cos_center0', 'eem_1', 'molecule_atom_1_dist_mean', 'dist_C_1_y', 'dist_H_0_y', 'dist_O_1_x', 'tertiary_atom_4', 'dist_C_3_y', 'dist_H_1_x', 'dist_N_0_x', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_4', 'dist_to_type_std', 'yukawa_H.y', 'atom_1_n_bonds', 'dist_C_0_x', 'molecule_type_dist_mean_diff', 'dist_xyz', 'atom_1_bond_lengths_max', 'cos_center1', 'dist_N_1_y', 'inv_dist1R', 'dist_O_0_x', 'distC0', 'molecule_couples', 'tertiary_angle_6', 'inv_dist0R', 'tertiary_atom_2', 'dist_to_type_1_mean', 'dist_C_3_x', 'coulomb_H.x', 'tertiary_angle_1', 'dist_N_0_y', 'tertiary_distance_3', 'tertiary_atom_3', 'tertiary_distance_1', 'molecule_dist_max', 'dist_C_0_y', 'dist_H_3_x', 'NH', 'dist_H_3_y', 'atom_index_farthest_0', 'eem_0', 'dist_C_1_x', 'tertiary_angle_2', 'inv_distPE', 'tertiary_distance_2', 'dist_C_4_x', 'tertiary_atom_1', 'mean_molecule_atom_1_dist_xyz', 'molecule_dist_mean', 'max_molecule_atom_0_dist_xyz', 'inv_dist1E', 'bond_atom', 'dist_H_0_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_angle_1', 'atom_1_bond_lengths_std', 'tertiary_angle_0', 'dist_C_0_x', 'dist_to_type_1_mean', 'inv_dist0R', 'adC1', 'dist_N_0_x', 'dist_H_0_y', 'inv_dist1R', 'dist_to_type_mean', 'min_molecule_atom_1_dist_xyz', 'cos_f0', 'tertiary_distance_0', 'tertiary_distance_1', 'tertiary_angle_2', 'dist_C_1_x', 'dist_C_2_y', 'eem_0', 'dist_H_0_x', 'dist_C_2_x', 'dist_O_0_y', 'inv_dist1', 'dist_C_0_y', 'dist_to_type_0_mean', 'eem_1', 'dist_H_1_x', 'dist_to_type_std', 'atom_1_bond_lengths_max', 'adC2', 'bond_atom', 'dist_O_0_x', 'dist_N_1_x', 'atom_1_n_bonds', 'molecule_atom_index_0_dist_max_div', 'vander_H.y', 'dist_H_2_y', 'atom_1_bond_lengths_mean', 'molecule_type_dist_mean', 'linkN', 'distN0', 'vander_H.x', 'tertiary_angle_3', 'dist_H_2_x', 'dist_C_1_y', 'linkM0', 'tertiary_distance_2', 'tertiary_distance_3', 'dist_N_2_x', 'dist_C_3_y', 'dist_N_0_y', 'dist_C_4_y', 'max_molecule_atom_0_dist_xyz', 'tertiary_atom_0', 'tertiary_atom_2', 'dist_C_3_x', 'molecule_type_dist_max', 'inv_distPE', 'atom_index_1_cycle_size_mean', 'tertiary_angle_4', 'vander_C.y', 'vander_O.x', 'tertiary_distance_5', 'yukawa_H.x', 'tertiary_distance_4', 'dist_H_1_y', 'cos_f1', 'dist_O_1_x', 'cos_center0', 'dist_xyz'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_angle_1', 'atom_1_bond_lengths_std', 'tertiary_angle_0', 'dist_C_0_x', 'dist_to_type_1_mean', 'inv_dist0R', 'adC1', 'dist_N_0_x', 'dist_H_0_y', 'inv_dist1R', 'dist_to_type_mean', 'min_molecule_atom_1_dist_xyz', 'cos_f0', 'tertiary_distance_0', 'tertiary_distance_1', 'tertiary_angle_2', 'dist_C_1_x', 'dist_C_2_y', 'eem_0', 'dist_H_0_x', 'dist_C_2_x', 'dist_O_0_y', 'inv_dist1', 'dist_C_0_y', 'dist_to_type_0_mean', 'eem_1', 'dist_H_1_x', 'dist_to_type_std', 'atom_1_bond_lengths_max', 'adC2', 'bond_atom', 'dist_O_0_x', 'dist_N_1_x', 'atom_1_n_bonds', 'molecule_atom_index_0_dist_max_div', 'vander_H.y', 'dist_H_2_y', 'atom_1_bond_lengths_mean', 'molecule_type_dist_mean', 'linkN', 'distN0', 'vander_H.x', 'tertiary_angle_3', 'dist_H_2_x', 'dist_C_1_y', 'linkM0', 'tertiary_distance_2', 'tertiary_distance_3', 'dist_N_2_x', 'dist_C_3_y', 'dist_N_0_y', 'dist_C_4_y', 'max_molecule_atom_0_dist_xyz', 'tertiary_atom_0', 'tertiary_atom_2', 'dist_C_3_x', 'molecule_type_dist_max', 'inv_distPE', 'atom_index_1_cycle_size_mean', 'tertiary_angle_4', 'vander_C.y', 'vander_O.x', 'tertiary_distance_5', 'yukawa_H.x', 'tertiary_distance_4', 'dist_H_1_y', 'cos_f1', 'dist_O_1_x', 'cos_center0', 'dist_xyz'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_O_0_y', 'dist_O_0_x', 'cos_c1', 'cos_c0_c1', 'dist_N_0_y', 'dist_N_0_x', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_0', 'cos_c0', 'adC3', 'dist_to_type_1_mean', 'dist_to_type_0_mean', 'dist_C_1_y', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'dist_xyz', 'dist_C_0_y', 'dist_to_type_mean', 'tertiary_atom_1', 'inv_distP', 'dist_C_2_y', 'tertiary_distance_2', 'adC1', 'inv_distPR', 'inv_distPE', 'inv_dist0R', 'yukawa_H.x', 'dist_C_1_x', 'dist_C_0_x', 'eem_0', 'adC2', 'tertiary_distance_1', 'dist_C_2_x', 'distance_c1', 'max_distance_y', 'eem_1', 'min_molecule_atom_1_dist_xyz', 'dist_H_0_y', 'tertiary_angle_1', 'dist_H_1_y', 'min_molecule_atom_0_dist_xyz', 'adC4', 'adN1', 'dist_C_3_y', 'dist_H_2_y', 'cos_f1', 'vander_N.x', 'cos_f0', 'coulomb_H.x', 'yukawa_H.y', 'tertiary_atom_2', 'distance_farthest_0', 'cos_f0_f1', 'tertiary_angle_3', 'dist_H_0_x', 'tertiary_atom_4', 'dist_H_3_y', 'molecule_dist_min', 'tertiary_angle_2', 'distH0', 'molecule_atom_index_0_dist_mean_div', 'tertiary_atom_0', 'tertiary_atom_5', 'dist_H_1_x', 'dist_H_2_x', 'linkM0', 'dist_H_3_x', 'inv_dist0E', 'dist_O_1_x', 'molecule_atom_index_1_dist_max_div', 'atom_index_farthest_1', 'tertiary_angle_4', 'atom_index_1', 'dist_C_3_x', 'tertiary_atom_3', 'tertiary_distance_3', 'vander_H.x', 'vander_H.y', 'linkN', 'coulomb_H.y', 'dist_H_4_y', 'dist_N_1_x', 'dist_O_1_y', 'molecule_dist_mean', 'tertiary_atom_6', 'tertiary_distance_5', 'vander_N.y', 'max_molecule_atom_0_dist_xyz', 'atom_index_farthest_0', 'molecule_dist_max', 'yukawa_C.x', 'NH', 'tertiary_distance_6', 'inv_dist0', 'mean_molecule_atom_0_dist_xyz', 'adN2', 'cos_center0_center1', 'dist_C_4_x', 'yukawa_C.y', 'dist_C_4_y', 'molecule_atom_index_0_dist_mean_diff', 'molecule_type_0_dist_std_diff', 'molecule_type_dist_min', 'dist_N_1_y', 'tertiary_angle_5', 'dist_H_4_x', 'tertiary_distance_4', 'molecule_atom_1_dist_min_diff', 'molecule_atom_index_1_dist_mean_diff', 'tertiary_distance_8', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_max', 'tertiary_distance_7', 'molecule_couples', 'link0', 'distance_center1', 'sd_molecule_atom_0_dist_xyz', 'distH1', 'molecule_atom_index_0_dist_max_diff', 'tertiary_angle_6'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_O_0_y', 'dist_O_0_x', 'cos_c1', 'cos_c0_c1', 'dist_N_0_y', 'dist_N_0_x', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_0', 'cos_c0', 'adC3', 'dist_to_type_1_mean', 'dist_to_type_0_mean', 'dist_C_1_y', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'dist_xyz', 'dist_C_0_y', 'dist_to_type_mean', 'tertiary_atom_1', 'inv_distP', 'dist_C_2_y', 'tertiary_distance_2', 'adC1', 'inv_distPR', 'inv_distPE', 'inv_dist0R', 'yukawa_H.x', 'dist_C_1_x', 'dist_C_0_x', 'eem_0', 'adC2', 'tertiary_distance_1', 'dist_C_2_x', 'distance_c1', 'max_distance_y', 'eem_1', 'min_molecule_atom_1_dist_xyz', 'dist_H_0_y', 'tertiary_angle_1', 'dist_H_1_y', 'min_molecule_atom_0_dist_xyz', 'adC4', 'adN1', 'dist_C_3_y', 'dist_H_2_y', 'cos_f1', 'vander_N.x', 'cos_f0', 'coulomb_H.x', 'yukawa_H.y', 'tertiary_atom_2', 'distance_farthest_0', 'cos_f0_f1', 'tertiary_angle_3', 'dist_H_0_x', 'tertiary_atom_4', 'dist_H_3_y', 'molecule_dist_min', 'tertiary_angle_2', 'distH0', 'molecule_atom_index_0_dist_mean_div', 'tertiary_atom_0', 'tertiary_atom_5', 'dist_H_1_x', 'dist_H_2_x', 'linkM0', 'dist_H_3_x', 'inv_dist0E', 'dist_O_1_x', 'molecule_atom_index_1_dist_max_div', 'atom_index_farthest_1', 'tertiary_angle_4', 'atom_index_1', 'dist_C_3_x', 'tertiary_atom_3', 'tertiary_distance_3', 'vander_H.x', 'vander_H.y', 'linkN', 'coulomb_H.y', 'dist_H_4_y', 'dist_N_1_x', 'dist_O_1_y', 'molecule_dist_mean', 'tertiary_atom_6', 'tertiary_distance_5', 'vander_N.y', 'max_molecule_atom_0_dist_xyz', 'atom_index_farthest_0', 'molecule_dist_max', 'yukawa_C.x', 'NH', 'tertiary_distance_6', 'inv_dist0', 'mean_molecule_atom_0_dist_xyz', 'adN2', 'cos_center0_center1', 'dist_C_4_x', 'yukawa_C.y', 'dist_C_4_y', 'molecule_atom_index_0_dist_mean_diff', 'molecule_type_0_dist_std_diff', 'molecule_type_dist_min', 'dist_N_1_y', 'tertiary_angle_5', 'dist_H_4_x', 'tertiary_distance_4', 'molecule_atom_1_dist_min_diff', 'molecule_atom_index_1_dist_mean_diff', 'tertiary_distance_8', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_max', 'tertiary_distance_7', 'molecule_couples', 'link0', 'distance_center1', 'sd_molecule_atom_0_dist_xyz', 'distH1', 'molecule_atom_index_0_dist_max_diff', 'tertiary_angle_6'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_O_0_y', 'dist_O_0_x', 'cos_c1', 'cos_c0_c1', 'dist_N_0_y', 'dist_N_0_x', 'molecule_atom_index_0_dist_min_diff', 'tertiary_angle_0', 'cos_c0', 'adC3', 'dist_to_type_1_mean', 'dist_to_type_0_mean', 'dist_C_1_y', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'dist_xyz', 'dist_C_0_y', 'dist_to_type_mean', 'tertiary_atom_1', 'inv_distP', 'dist_C_2_y', 'tertiary_distance_2', 'adC1', 'inv_distPR', 'inv_distPE', 'inv_dist0R', 'yukawa_H.x', 'dist_C_1_x', 'dist_C_0_x', 'eem_0', 'adC2', 'tertiary_distance_1', 'dist_C_2_x', 'distance_c1', 'max_distance_y', 'eem_1', 'min_molecule_atom_1_dist_xyz', 'dist_H_0_y', 'tertiary_angle_1', 'dist_H_1_y', 'min_molecule_atom_0_dist_xyz', 'adC4', 'adN1', 'dist_C_3_y', 'dist_H_2_y', 'cos_f1', 'vander_N.x', 'cos_f0', 'coulomb_H.x', 'yukawa_H.y', 'tertiary_atom_2', 'distance_farthest_0', 'cos_f0_f1', 'tertiary_angle_3', 'dist_H_0_x', 'tertiary_atom_4', 'dist_H_3_y', 'molecule_dist_min', 'tertiary_angle_2', 'distH0', 'molecule_atom_index_0_dist_mean_div', 'tertiary_atom_0', 'tertiary_atom_5', 'dist_H_1_x', 'dist_H_2_x', 'linkM0', 'dist_H_3_x', 'inv_dist0E', 'dist_O_1_x', 'molecule_atom_index_1_dist_max_div', 'atom_index_farthest_1', 'tertiary_angle_4', 'atom_index_1', 'dist_C_3_x', 'tertiary_atom_3', 'tertiary_distance_3', 'vander_H.x', 'vander_H.y', 'linkN', 'coulomb_H.y', 'dist_H_4_y', 'dist_N_1_x', 'dist_O_1_y', 'molecule_dist_mean', 'tertiary_atom_6', 'tertiary_distance_5', 'vander_N.y', 'max_molecule_atom_0_dist_xyz', 'atom_index_farthest_0', 'molecule_dist_max', 'yukawa_C.x', 'NH', 'tertiary_distance_6', 'inv_dist0', 'mean_molecule_atom_0_dist_xyz', 'adN2', 'cos_center0_center1', 'dist_C_4_x', 'yukawa_C.y', 'dist_C_4_y', 'molecule_atom_index_0_dist_mean_diff', 'molecule_type_0_dist_std_diff', 'molecule_type_dist_min', 'dist_N_1_y', 'tertiary_angle_5', 'dist_H_4_x', 'tertiary_distance_4', 'molecule_atom_1_dist_min_diff', 'molecule_atom_index_1_dist_mean_diff', 'tertiary_distance_8', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_max', 'tertiary_distance_7', 'molecule_couples', 'link0', 'distance_center1', 'sd_molecule_atom_0_dist_xyz', 'distH1', 'molecule_atom_index_0_dist_max_diff', 'tertiary_angle_6'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['cos_c0', 'molecule_atom_index_0_dist_min_diff', 'dist_C_1_x', 'molecule_atom_index_0_dist_std_diff', 'dist_N_0_y', 'dist_to_type_mean', 'tertiary_angle_4', 'max_molecule_atom_1_dist_xyz', 'tertiary_atom_0', 'tertiary_distance_3', 'dist_to_type_0_mean', 'dist_C_0_x', 'tertiary_distance_1', 'distance_c1', 'vander_C.y', 'tertiary_angle_1', 'dist_C_1_y', 'dist_O_0_y', 'cos_f1', 'dist_H_0_x', 'adC2', 'dist_H_0_y', 'molecule_atom_index_0_dist_std_div', 'tertiary_atom_2', 'cos_c1', 'dist_xyz', 'inv_dist1R', 'min_molecule_atom_0_dist_xyz', 'dist_to_type_std', 'tertiary_distance_4', 'tertiary_atom_1', 'dist_C_0_y', 'tertiary_angle_0', 'inv_dist0R', 'inv_distPE', 'vander_C.x', 'eem_1', 'cos_f0_f1', 'distance_farthest_0', 'molecule_atom_1_dist_min_diff', 'cos_f0', 'tertiary_angle_2', 'adC3', 'dist_O_0_x', 'dist_N_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_H_1_y', 'atom_1_bond_lengths_mean', 'tertiary_angle_3', 'distH0', 'dist_C_2_y', 'adC1', 'dist_to_type_1_mean', 'tertiary_distance_2', 'max_distance_y', 'cos_c0_c1', 'dist_C_2_x', 'dist_C_3_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_3', 'tertiary_atom_1', 'dist_C_4_x', 'bond_atom', 'atom_index_1_cycle_size_mean', 'tertiary_angle_3', 'molecule_atom_index_0_dist_min_div', 'dist_C_2_x', 'distN0', 'cos_c0_c1', 'dist_to_type_mean', 'atom_1_bond_lengths_min', 'dist_N_1_x', 'dist_O_0_y', 'adC2', 'molecule_atom_index_1_dist_min_div', 'cos_c0', 'dist_C_1_x', 'tertiary_angle_2', 'dist_to_type_std', 'dist_C_2_y', 'atom_1_bond_lengths_max', 'tertiary_atom_3', 'cos_f0', 'molecule_atom_index_0_dist_mean_diff', 'dist_C_1_y', 'dist_O_0_x', 'max_molecule_atom_0_dist_xyz', 'tertiary_distance_1', 'molecule_atom_index_0_dist_min_diff', 'tertiary_distance_2', 'dist_C_0_x', 'eem_1', 'atom_1_bond_lengths_mean', 'atom_1_bond_lengths_std', 'inv_dist0R', 'inv_distPR', 'dist_N_0_y', 'atom_1_n_bonds', 'tertiary_angle_1', 'dist_C_0_y', 'inv_dist1R', 'dist_C_3_x', 'tertiary_angle_0', 'cos_c1', 'dist_to_type_0_mean', 'adC1', 'molecule_atom_index_1_dist_min_diff', 'molecule_type_0_dist_std_diff'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}]

In [9]:
fc_columns_list = [{'columns': ['atom_index_1_cycle_size_mean', 'dist_H_0_y', 'dist_H_1_x', 'tertiary_atom_1', 'adC2', 'tertiary_distance_1', 'dist_C_2_y', 'molecule_atom_index_1_dist_min_diff', 'dist_C_1_y', 'atom_index_farthest_0', 'yukawa_H.y', 'inv_dist1R', 'dist_C_0_x', 'dist_H_3_x', 'tertiary_angle_3', 'dist_to_type_std', 'dist_C_3_y', 'eem_1', 'inv_distPR', 'dist_C_1_x', 'linkM0', 'dist_H_2_y', 'dist_H_0_x', 'adC1', 'tertiary_angle_0', 'adN1', 'dist_O_1_y', 'dist_C_2_x', 'eem_0', 'dist_C_4_x', 'adC3', 'tertiary_distance_3', 'dist_O_0_x', 'tertiary_angle_2', 'atom_1_bond_lengths_std', 'molecule_atom_index_0_dist_max_div', 'inv_dist0R', 'distance_farthest_0', 'yukawa_N.x', 'dist_H_3_y', 'molecule_type_dist_mean_diff', 'dist_N_0_x', 'max_molecule_atom_1_dist_xyz', 'dist_C_0_y', 'dist_H_2_x', 'dist_to_type_0_mean', 'tertiary_distance_5', 'atom_1_n_bonds', 'dist_C_3_x', 'tertiary_distance_0', 'dist_to_type_mean', 'distC1', 'dist_O_0_y', 'dist_C_4_y', 'tertiary_distance_2', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_N_0_y', 'dist_xyz', 'dist_H_1_y', 'dist_O_1_x', 'tertiary_distance_4', 'atom_1_bond_lengths_mean', 'yukawa_H.x', 'tertiary_atom_0', 'tertiary_angle_4', 'inv_dist1E', 'tertiary_angle_1'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
                {'columns': ['atom_index_1_cycle_size_mean', 'dist_H_0_y', 'dist_H_1_x', 'linkM1', 'adC2', 'tertiary_distance_1', 'tertiary_distance_6', 'dist_C_2_y', 'distance_center0', 'dist_C_1_y', 'yukawa_H.y', 'inv_dist1R', 'dist_C_0_x', 'yukawa_C.x', 'inv_dist1', 'cos_center0', 'tertiary_angle_3', 'cos_center1', 'dist_C_3_y', 'vander_O.y', 'eem_1', 'inv_distPR', 'dist_C_1_x', 'inv_distP', 'vander_N.y', 'dist_H_2_y', 'dist_H_0_x', 'tertiary_angle_0', 'vander_N.x', 'dist_C_2_x', 'eem_0', 'coulomb_H.x', 'min_molecule_atom_1_dist_xyz', 'yukawa_N.y', 'molecule_atom_index_0_dist_mean_div', 'dist_C_4_x', 'adC3', 'tertiary_distance_3', 'linkN', 'dist_O_0_x', 'max_molecule_atom_0_dist_xyz', 'distance_center1', 'tertiary_angle_2', 'atom_1_bond_lengths_std', 'molecule_atom_index_0_dist_max_div', 'inv_dist0R', 'link0', 'yukawa_N.x', 'dist_H_3_y', 'dist_N_0_x', 'dist_C_0_y', 'molecule_atom_1_dist_min_diff', 'dist_to_type_0_mean', 'inv_distPE', 'vander_H.x', 'tertiary_distance_5', 'dist_N_1_y', 'dist_N_1_x', 'dist_C_3_x', 'tertiary_distance_0', 'dist_O_0_y', 'distN0', 'dist_C_4_y', 'cos_f1', 'tertiary_distance_2', 'vander_O.x', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_N_0_y', 'dist_H_1_y', 'tertiary_distance_4', 'atom_1_bond_lengths_mean', 'tertiary_atom_0', 'distN1', 'tertiary_angle_1', 'adC4', 'bond_atom'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
                {'columns': ['atom_index_1_cycle_size_mean', 'atom_0_bond_lengths_max', 'dist_H_0_y', 'tertiary_atom_1', 'adC2', 'tertiary_atom_2', 'tertiary_distance_6', 'tertiary_distance_1', 'dist_C_2_y', 'molecule_atom_index_1_dist_min_diff', 'dist_C_1_y', 'dist_C_0_x', 'inv_dist1R', 'atom_1_bond_lengths_min', 'cos_c0_c1', 'tertiary_angle_3', 'inv_dist1', 'molecule_atom_index_0_dist_max_diff', 'dist_C_3_y', 'vander_O.y', 'eem_1', 'inv_distPR', 'dist_C_1_x', 'dist_H_0_x', 'adC1', 'tertiary_angle_0', 'adN1', 'dist_C_2_x', 'molecule_atom_index_0_dist_mean_div', 'molecule_atom_index_0_dist_min_diff', 'adC3', 'tertiary_distance_3', 'dist_O_0_x', 'max_molecule_atom_0_dist_xyz', 'tertiary_angle_2', 'atom_1_bond_lengths_std', 'inv_dist0R', 'distance_farthest_0', 'dist_N_0_x', 'dist_C_0_y', 'atom_index_1_n_cycle', 'molecule_atom_1_dist_min_diff', 'dist_to_type_0_mean', 'cos_c0', 'distC0', 'tertiary_atom_3', 'bond_atom', 'molecule_atom_index_1_dist_min_div', 'atom_1_n_bonds', 'vander_C.y', 'dist_C_3_x', 'tertiary_atom_4', 'dist_O_0_y', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_2', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_N_0_y', 'dist_xyz', 'dist_H_1_y', 'dist_O_1_x', 'atom_1_bond_lengths_mean', 'min_molecule_atom_0_dist_xyz', 'molecule_type_dist_max', 'tertiary_atom_0', 'tertiary_angle_4', 'tertiary_angle_1', 'cos_c1'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
                {'columns': ['yukawa_H.y', 'dist_H_0_y', 'dist_H_3_x', 'tertiary_distance_2', 'dist_H_1_y', 'tertiary_angle_0', 'vander_H.y', 'adC3', 'dist_H_1_x', 'tertiary_distance_3', 'tertiary_atom_2', 'dist_to_type_0_mean', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_3', 'dist_H_2_x', 'dist_to_type_std', 'dist_C_0_y', 'tertiary_distance_1', 'adN1', 'molecule_atom_index_0_dist_mean_diff', 'inv_distPE', 'adC1', 'molecule_type_dist_max', 'cos_f0', 'eem_1', 'tertiary_atom_1', 'dist_C_3_y', 'molecule_atom_index_0_dist_min_diff', 'dist_N_0_y', 'cos_c1', 'molecule_dist_min', 'vander_H.x', 'mean_molecule_atom_1_dist_xyz', 'dist_H_2_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'molecule_atom_index_0_dist_max_div', 'dist_C_3_x', 'cos_c0', 'mean_molecule_atom_0_dist_xyz', 'cos_f0_f1', 'cos_c0_c1', 'inv_distPR', 'dist_N_0_x', 'dist_C_1_x', 'dist_H_4_x', 'dist_O_0_x', 'dist_H_4_y', 'adC2', 'cos_f1', 'dist_C_0_x', 'min_molecule_atom_1_dist_xyz', 'dist_H_3_y', 'dist_C_1_y', 'max_distance_y', 'dist_C_2_y', 'dist_to_type_mean', 'yukawa_H.x', 'dist_xyz', 'dist_H_0_x', 'distance_c1', 'inv_distP', 'link0', 'tertiary_distance_4', 'tertiary_angle_1', 'distance_farthest_0', 'dist_C_2_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
                {'columns': ['yukawa_H.y', 'dist_H_0_y', 'dist_H_3_x', 'tertiary_distance_2', 'dist_H_1_y', 'tertiary_angle_0', 'vander_H.y', 'adC3', 'dist_H_1_x', 'tertiary_distance_3', 'tertiary_atom_2', 'dist_to_type_0_mean', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_3', 'dist_H_2_x', 'dist_to_type_std', 'dist_C_0_y', 'tertiary_distance_1', 'adN1', 'molecule_atom_index_0_dist_mean_diff', 'inv_distPE', 'adC1', 'molecule_type_dist_max', 'cos_f0', 'eem_1', 'tertiary_atom_1', 'dist_C_3_y', 'molecule_atom_index_0_dist_min_diff', 'dist_N_0_y', 'cos_c1', 'molecule_dist_min', 'vander_H.x', 'mean_molecule_atom_1_dist_xyz', 'dist_H_2_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'molecule_atom_index_0_dist_max_div', 'dist_C_3_x', 'cos_c0', 'mean_molecule_atom_0_dist_xyz', 'cos_f0_f1', 'cos_c0_c1', 'inv_distPR', 'dist_N_0_x', 'dist_C_1_x', 'dist_H_4_x', 'dist_O_0_x', 'dist_H_4_y', 'adC2', 'cos_f1', 'dist_C_0_x', 'min_molecule_atom_1_dist_xyz', 'dist_H_3_y', 'dist_C_1_y', 'max_distance_y', 'dist_C_2_y', 'dist_to_type_mean', 'yukawa_H.x', 'dist_xyz', 'dist_H_0_x', 'distance_c1', 'inv_distP', 'link0', 'tertiary_distance_4', 'tertiary_angle_1', 'distance_farthest_0', 'dist_C_2_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
                {'columns': ['tertiary_distance_6', 'dist_C_4_y', 'molecule_atom_index_0_dist_std_diff', 'inv_dist1R', 'molecule_atom_index_1_dist_min_div', 'tertiary_distance_3', 'atom_1_n_bonds', 'molecule_atom_index_0_dist_max_div', 'tertiary_atom_2', 'vander_O.x', 'atom_1_bond_lengths_max', 'dist_O_0_y', 'distance_farthest_0', 'dist_to_type_std', 'molecule_atom_1_dist_min_div', 'adC1', 'adC2', 'molecule_atom_index_1_dist_min_diff', 'adN1', 'dist_N_0_y', 'atom_index_1_n_cycle', 'inv_dist1E', 'molecule_type_0_dist_std_diff', 'inv_dist0', 'molecule_atom_index_0_dist_min_diff', 'mean_molecule_atom_1_dist_xyz', 'atom_index_1_cycle_size_mean', 'dist_C_0_y', 'dist_xyz', 'tertiary_angle_4', 'dist_H_1_x', 'atom_0_bond_lengths_max', 'molecule_type_dist_mean_div', 'tertiary_angle_3', 'cos_c1', 'tertiary_distance_4', 'atom_1_bond_lengths_mean', 'dist_C_3_x', 'max_molecule_atom_0_dist_xyz', 'tertiary_angle_1', 'molecule_atom_index_1_dist_std_diff', 'dist_C_0_x', 'dist_N_0_x', 'dist_O_1_y', 'dist_H_0_x', 'cos_f0', 'tertiary_atom_1', 'vander_O.y', 'tertiary_angle_2', 'eem_0', 'dist_C_2_y', 'adC3', 'molecule_atom_1_dist_min_diff', 'tertiary_distance_1', 'dist_O_1_x', 'dist_to_type_mean', 'tertiary_distance_5', 'dist_C_1_x', 'dist_H_1_y', 'dist_H_0_y', 'molecule_atom_index_0_dist_min_div', 'dist_H_2_y', 'tertiary_distance_2', 'molecule_type_dist_std_diff', 'eem_1', 'dist_C_2_x', 'distC0', 'molecule_atom_index_0_dist_max_diff', 'inv_dist1', 'dist_C_1_y', 'tertiary_angle_0', 'dist_O_0_x', 'link0', 'mean_molecule_atom_0_dist_xyz', 'dist_C_3_y', 'tertiary_atom_3', 'cos_c0_c1', 'cos_c0', 'inv_dist0R'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
                {'columns': ['cos_c0_c1', 'tertiary_angle_0', 'cos_c0', 'cos_c1', 'max_molecule_atom_1_dist_xyz', 'molecule_atom_index_0_dist_min_diff', 'dist_H_0_y', 'dist_O_0_y', 'dist_O_0_x', 'dist_C_2_y', 'dist_C_1_y', 'tertiary_angle_1', 'dist_C_0_y', 'molecule_atom_index_1_dist_max_div', 'tertiary_atom_1', 'tertiary_distance_1', 'dist_C_0_x', 'tertiary_distance_3', 'adC3', 'molecule_atom_index_0_dist_min_div', 'max_molecule_atom_0_dist_xyz', 'tertiary_angle_3', 'inv_distPR', 'tertiary_distance_2', 'atom_index_closest_1', 'dist_N_0_y', 'tertiary_angle_2', 'dist_to_type_mean', 'dist_C_2_x', 'distance_c1', 'dist_H_1_y', 'dist_H_0_x', 'cos_f0', 'dist_C_1_x', 'adC2', 'eem_1', 'dist_to_type_0_mean', 'dist_to_type_std', 'max_distance_y', 'distH0', 'atom_1_bond_lengths_mean', 'cos_f1', 'tertiary_atom_0', 'dist_C_3_y', 'dist_xyz', 'dist_to_type_1_mean', 'dist_C_3_x', 'inv_dist0R', 'adN1', 'tertiary_atom_2', 'molecule_atom_index_1_dist_max_diff', 'dist_N_0_x', 'mean_molecule_atom_0_dist_xyz', 'dist_H_2_y', 'dist_H_1_x', 'linkN', 'cos_f0_f1', 'adC1', 'adH1', 'tertiary_angle_4', 'adH2', 'molecule_type_dist_std_diff', 'yukawa_C.y', 'dist_O_1_x', 'linkM0', 'molecule_atom_index_0_dist_max_div', 'link0', 'molecule_atom_index_0_dist_max_diff', 'dist_H_2_x', 'dist_C_4_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
                {'columns': ['tertiary_angle_1', 'dist_C_4_y', 'molecule_atom_index_0_dist_std_diff', 'inv_dist1R', 'dist_C_0_x', 'tertiary_distance_3', 'dist_N_0_x', 'molecule_atom_index_1_dist_max_div', 'atom_1_n_bonds', 'dist_H_0_x', 'tertiary_atom_2', 'cos_f0', 'tertiary_atom_1', 'atom_1_bond_lengths_max', 'dist_O_0_y', 'dist_to_type_std', 'vander_O.y', 'tertiary_angle_2', 'yukawa_H.y', 'dist_C_2_y', 'adC3', 'adC1', 'tertiary_distance_1', 'adC2', 'dist_H_0_y', 'dist_C_1_x', 'molecule_atom_index_1_dist_min_diff', 'dist_H_1_y', 'bond_atom', 'atom_1_bond_lengths_std', 'molecule_atom_index_0_dist_min_div', 'dist_H_2_y', 'dist_N_0_y', 'tertiary_distance_2', 'molecule_atom_index_0_dist_min_diff', 'eem_1', 'atom_index_1_cycle_size_mean', 'dist_C_0_y', 'dist_C_2_x', 'dist_xyz', 'dist_H_1_x', 'dist_C_4_x', 'molecule_atom_index_0_dist_std_div', 'dist_C_1_y', 'tertiary_angle_0', 'dist_O_0_x', 'dist_to_type_1_mean', 'inv_distPR', 'atom_1_bond_lengths_min', 'min_molecule_atom_0_dist_xyz', 'dist_C_3_y', 'tertiary_angle_3', 'cos_c1', 'tertiary_distance_4', 'tertiary_atom_3', 'atom_1_bond_lengths_mean', 'dist_C_3_x', 'dist_H_3_y', 'cos_c0_c1', 'cos_c0', 'inv_dist0R', 'max_molecule_atom_0_dist_xyz'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
               ]

In [10]:
dso_columns_list = [
{'columns': ['dist_N_0_y', 'tertiary_distance_6', 'tertiary_distance_9', 'dist_H_2_y', 'dist_C_1_y', 'tertiary_distance_11', 'dist_H_3_y', 'dist_O_0_x', 'inv_dist1', 'yukawa_F.x', 'yukawa_N.y', 'dist_O_1_x', 'yukawa_N.x', 'tertiary_distance_10', 'inv_dist0R', 'tertiary_atom_0', 'vander_C.y', 'eem_1', 'vander_O.x', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_N_0_x', 'dist_H_4_y', 'tertiary_distance_4', 'dist_N_1_x', 'yukawa_O.y', 'atom_n', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_bond_lengths_mean', 'yukawa_C.y', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'tertiary_atom_2', 'dist_O_0_y', 'vander_H.y', 'tertiary_distance_7', 'dist_O_1_y', 'tertiary_distance_0', 'tertiary_distance_2', 'dist_N_1_y', 'atom_1_bond_lengths_std', 'dist_C_0_y', 'dist_C_4_y', 'vander_O.y', 'dist_C_1_x', 'tertiary_distance_5', 'dist_C_3_y', 'tertiary_distance_1', 'tertiary_distance_8', 'tertiary_angle_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_6', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_9', 'dist_C_1_y', 'dist_C_3_x', 'yukawa_N.x', 'dist_H_3_y', 'inv_dist1', 'yukawa_F.x', 'inv_dist0R', 'vander_C.y', 'tertiary_atom_0', 'eem_1', 'bond_atom', 'vander_O.x', 'molecule_name.1', 'tertiary_distance_4', 'dist_N_1_x', 'mean_molecule_atom_0_dist_xyz', 'yukawa_O.y', 'inv_dist0E', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'tertiary_angle_3', 'inv_dist0', 'dist_O_0_y', 'vander_H.y', 'coulomb_H.y', 'tertiary_distance_2', 'tertiary_distance_3', 'tertiary_distance_0', 'distance_center1', 'dist_O_1_y', 'dist_H_0_x', 'dist_C_0_y', 'tertiary_angle_0', 'tertiary_distance_5', 'tertiary_distance_1', 'dist_C_3_y', 'molecule_atom_index_1_dist_std_diff', 'tertiary_angle_2', 'dist_N_0_y', 'eem_0', 'dist_H_2_y', 'vander_F.y', 'dist_C_2_x', 'tertiary_atom_14', 'dist_H_4_x', 'tertiary_distance_11', 'mean_molecule_atom_1_dist_xyz', 'dist_O_0_x', 'yukawa_N.y', 'dist_H_1_x', 'tertiary_distance_10', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_N_2_x', 'dist_N_0_x', 'distance_center0', 'atom_1_bond_lengths_mean', 'dist_H_0_y', 'yukawa_C.y', 'tertiary_atom_2', 'tertiary_distance_7', 'vander_C.x', 'tertiary_angle_1', 'atom_1_bond_lengths_std', 'dist_C_4_y', 'dist_C_1_x', 'tertiary_angle_4', 'vander_N.x', 'tertiary_distance_8'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_6', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_9', 'dist_C_1_y', 'dist_C_3_x', 'yukawa_N.x', 'dist_H_3_y', 'inv_dist1', 'yukawa_F.x', 'inv_dist0R', 'vander_C.y', 'tertiary_atom_0', 'eem_1', 'bond_atom', 'vander_O.x', 'molecule_name.1', 'tertiary_distance_4', 'dist_N_1_x', 'mean_molecule_atom_0_dist_xyz', 'yukawa_O.y', 'inv_dist0E', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'tertiary_angle_3', 'inv_dist0', 'dist_O_0_y', 'vander_H.y', 'coulomb_H.y', 'tertiary_distance_2', 'tertiary_distance_3', 'tertiary_distance_0', 'distance_center1', 'dist_O_1_y', 'dist_H_0_x', 'dist_C_0_y', 'tertiary_angle_0', 'tertiary_distance_5', 'tertiary_distance_1', 'dist_C_3_y', 'molecule_atom_index_1_dist_std_diff', 'tertiary_angle_2', 'dist_N_0_y', 'eem_0', 'dist_H_2_y', 'vander_F.y', 'dist_C_2_x', 'tertiary_atom_14', 'dist_H_4_x', 'tertiary_distance_11', 'mean_molecule_atom_1_dist_xyz', 'dist_O_0_x', 'yukawa_N.y', 'dist_H_1_x', 'tertiary_distance_10', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_N_2_x', 'dist_N_0_x', 'distance_center0', 'atom_1_bond_lengths_mean', 'dist_H_0_y', 'yukawa_C.y', 'tertiary_atom_2', 'tertiary_distance_7', 'vander_C.x', 'tertiary_angle_1', 'atom_1_bond_lengths_std', 'dist_C_4_y', 'dist_C_1_x', 'tertiary_angle_4', 'vander_N.x', 'tertiary_distance_8'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_6', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_9', 'dist_C_1_y', 'dist_C_3_x', 'yukawa_N.x', 'dist_H_3_y', 'inv_dist1', 'yukawa_F.x', 'inv_dist0R', 'vander_C.y', 'tertiary_atom_0', 'eem_1', 'bond_atom', 'vander_O.x', 'molecule_name.1', 'tertiary_distance_4', 'dist_N_1_x', 'mean_molecule_atom_0_dist_xyz', 'yukawa_O.y', 'inv_dist0E', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'tertiary_angle_3', 'inv_dist0', 'dist_O_0_y', 'vander_H.y', 'coulomb_H.y', 'tertiary_distance_2', 'tertiary_distance_3', 'tertiary_distance_0', 'distance_center1', 'dist_O_1_y', 'dist_H_0_x', 'dist_C_0_y', 'tertiary_angle_0', 'tertiary_distance_5', 'tertiary_distance_1', 'dist_C_3_y', 'molecule_atom_index_1_dist_std_diff', 'tertiary_angle_2', 'dist_N_0_y', 'eem_0', 'dist_H_2_y', 'vander_F.y', 'dist_C_2_x', 'tertiary_atom_14', 'dist_H_4_x', 'tertiary_distance_11', 'mean_molecule_atom_1_dist_xyz', 'dist_O_0_x', 'yukawa_N.y', 'dist_H_1_x', 'tertiary_distance_10', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_N_2_x', 'dist_N_0_x', 'distance_center0', 'atom_1_bond_lengths_mean', 'dist_H_0_y', 'yukawa_C.y', 'tertiary_atom_2', 'tertiary_distance_7', 'vander_C.x', 'tertiary_angle_1', 'atom_1_bond_lengths_std', 'dist_C_4_y', 'dist_C_1_x', 'tertiary_angle_4', 'vander_N.x', 'tertiary_distance_8'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['inv_distPR', 'min_molecule_atom_0_dist_xyz', 'tertiary_distance_6', 'tertiary_distance_9', 'dist_C_1_y', 'tertiary_distance_15', 'distance_farthest_0', 'dist_to_type_mean', 'dist_C_0_x', 'molecule_atom_index_1_dist_mean_div', 'adN1', 'yukawa_F.y', 'dist_H_3_y', 'yukawa_N.x', 'inv_dist0R', 'vander_C.y', 'eem_1', 'bond_atom', 'vander_O.x', 'molecule_name.1', 'tertiary_distance_4', 'cos_c0', 'dist_N_1_x', 'mean_molecule_atom_0_dist_xyz', 'yukawa_O.y', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_n_bonds', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'atom_1_bond_lengths_min', 'tertiary_angle_3', 'dist_O_0_y', 'coulomb_H.y', 'distance_center1', 'tertiary_distance_2', 'dist_O_1_y', 'adC2', 'dist_N_1_y', 'tertiary_distance_3', 'dist_H_0_x', 'dist_to_type_0_mean', 'dist_C_4_x', 'dist_C_0_y', 'cos_c0_c1', 'tertiary_angle_0', 'adC3', 'molecule_couples', 'tertiary_distance_5', 'tertiary_distance_1', 'molecule_atom_index_1_dist_min_diff', 'dist_C_3_y', 'tertiary_angle_2', 'distN0', 'dist_N_0_y', 'molecule_atom_index_0_dist_min_diff', 'dist_H_2_y', 'dist_C_2_x', 'vander_N.y', 'tertiary_distance_11', 'dist_O_0_x', 'cos_c1', 'yukawa_N.y', 'dist_H_1_x', 'tertiary_distance_10', 'cos_f0', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_H_4_y', 'molecule_atom_index_0_dist_mean_div', 'yukawa_C.y', 'tertiary_distance_12', 'molecule_atom_index_0_dist_min_div', 'inv_dist1R', 'dist_xyz', 'tertiary_distance_7', 'linkN', 'vander_C.x', 'tertiary_atom_1', 'tertiary_angle_1', 'atom_1_bond_lengths_std', 'dist_C_4_y', 'vander_O.y', 'dist_to_type_std', 'dist_C_1_x', 'tertiary_angle_4', 'dist_C_3_x', 'tertiary_distance_8'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['inv_distPR', 'min_molecule_atom_0_dist_xyz', 'tertiary_distance_6', 'tertiary_distance_9', 'dist_C_1_y', 'tertiary_distance_15', 'distance_farthest_0', 'dist_to_type_mean', 'dist_C_0_x', 'molecule_atom_index_1_dist_mean_div', 'adN1', 'yukawa_F.y', 'dist_H_3_y', 'yukawa_N.x', 'inv_dist0R', 'vander_C.y', 'eem_1', 'bond_atom', 'vander_O.x', 'molecule_name.1', 'tertiary_distance_4', 'cos_c0', 'dist_N_1_x', 'mean_molecule_atom_0_dist_xyz', 'yukawa_O.y', 'dist_H_1_y', 'tertiary_distance_13', 'atom_1_n_bonds', 'atom_1_bond_lengths_max', 'yukawa_C.x', 'dist_C_2_y', 'atom_1_bond_lengths_min', 'tertiary_angle_3', 'dist_O_0_y', 'coulomb_H.y', 'distance_center1', 'tertiary_distance_2', 'dist_O_1_y', 'adC2', 'dist_N_1_y', 'tertiary_distance_3', 'dist_H_0_x', 'dist_to_type_0_mean', 'dist_C_4_x', 'dist_C_0_y', 'cos_c0_c1', 'tertiary_angle_0', 'adC3', 'molecule_couples', 'tertiary_distance_5', 'tertiary_distance_1', 'molecule_atom_index_1_dist_min_diff', 'dist_C_3_y', 'tertiary_angle_2', 'distN0', 'dist_N_0_y', 'molecule_atom_index_0_dist_min_diff', 'dist_H_2_y', 'dist_C_2_x', 'vander_N.y', 'tertiary_distance_11', 'dist_O_0_x', 'cos_c1', 'yukawa_N.y', 'dist_H_1_x', 'tertiary_distance_10', 'cos_f0', 'yukawa_O.x', 'atom_index_1_cycle_size_mean', 'dist_H_4_y', 'molecule_atom_index_0_dist_mean_div', 'yukawa_C.y', 'tertiary_distance_12', 'molecule_atom_index_0_dist_min_div', 'inv_dist1R', 'dist_xyz', 'tertiary_distance_7', 'linkN', 'vander_C.x', 'tertiary_atom_1', 'tertiary_angle_1', 'atom_1_bond_lengths_std', 'dist_C_4_y', 'vander_O.y', 'dist_to_type_std', 'dist_C_1_x', 'tertiary_angle_4', 'dist_C_3_x', 'tertiary_distance_8'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['tertiary_distance_6', 'dist_C_4_x', 'dist_C_3_x', 'cos_c1', 'yukawa_O.x', 'coulomb_H.y', 'molecule_atom_index_0_dist_min_diff', 'cos_c0', 'vander_O.y', 'dist_O_0_y', 'molecule_name.1', 'inv_distPR', 'tertiary_distance_18', 'dist_H_0_x', 'dist_O_2_y', 'yukawa_N.x', 'dist_to_type_std', 'adC1', 'dist_C_4_y', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'tertiary_distance_14', 'cos_center0_center1', 'dist_to_type_0_mean', 'tertiary_distance_11', 'vander_O.x', 'tertiary_distance_3', 'dist_H_0_y', 'dist_C_3_y', 'dist_C_0_y', 'dist_N_0_y', 'dist_N_1_y', 'dist_O_1_x', 'dist_H_4_x', 'dist_C_2_y', 'dist_O_1_y', 'adN1', 'molecule_dist_mean', 'molecule_atom_index_0_dist_mean_div', 'min_molecule_atom_0_dist_xyz', 'dist_N_1_x', 'dist_O_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_C_2_x', 'distance_c1', 'dist_C_1_y', 'yukawa_C.x', 'tertiary_distance_13', 'dist_to_type_1_mean', 'dist_xyz', 'dist_F_1_y', 'tertiary_distance_12', 'tertiary_atom_16', 'vander_C.x', 'dist_N_0_x', 'tertiary_distance_9', 'tertiary_distance_1', 'molecule_atom_1_dist_min_diff', 'tertiary_distance_16', 'yukawa_O.y', 'yukawa_N.y', 'tertiary_atom_17', 'tertiary_atom_15', 'tertiary_distance_7', 'link0', 'tertiary_distance_2', 'tertiary_angle_2', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'yukawa_C.y', 'dist_to_type_mean', 'tertiary_atom_18', 'tertiary_angle_1', 'tertiary_distance_5', 'cos_c0_c1', 'tertiary_distance_10', 'tertiary_distance_8', 'dist_C_1_x', 'vander_C.y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_C_4_x', 'dist_C_3_x', 'cos_c1', 'molecule_atom_index_0_dist_min_diff', 'cos_c0', 'dist_O_0_y', 'molecule_name.1', 'dist_H_1_y', 'dist_H_1_x', 'dist_H_2_x', 'dist_to_type_std', 'adC1', 'atom_0_bond_lengths_max', 'dist_C_4_y', 'tertiary_angle_3', 'max_molecule_atom_0_dist_xyz', 'dist_to_type_0_mean', 'tertiary_distance_3', 'atom_1_bond_lengths_std', 'dist_H_0_y', 'dist_C_3_y', 'dist_C_0_y', 'dist_N_0_y', 'dist_N_1_y', 'dist_C_2_y', 'dist_O_1_y', 'adC3', 'distance_center1', 'dist_O_0_x', 'atom_index_1_cycle_size_mean', 'tertiary_angle_5', 'tertiary_angle_12', 'adC2', 'dist_C_2_x', 'tertiary_distance_15', 'molecule_atom_index_1_dist_mean_diff', 'dist_C_1_y', 'tertiary_distance_13', 'atom_1_n_bonds', 'dist_to_type_1_mean', 'inv_dist1R', 'dist_xyz', 'tertiary_angle_14', 'tertiary_angle_13', 'dist_N_0_x', 'tertiary_distance_1', 'tertiary_angle_6', 'link1', 'tertiary_atom_0', 'molecule_atom_index_1_dist_min_div', 'tertiary_distance_2', 'tertiary_distance_7', 'tertiary_angle_2', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'atom_1_bond_lengths_min', 'atom_1_bond_lengths_max', 'cos_f0', 'dist_to_type_mean', 'tertiary_angle_1', 'tertiary_distance_5', 'cos_c0_c1', 'tertiary_angle_15', 'tertiary_angle_4'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
]

In [11]:
# columns_ = []

# for columns_list in [scalar_coupling_constant_columns_list, pso_columns_list,sd_columns_list,fc_columns_list,dso_columns_list]:
#     for columns in columns_list:
#         columns_ += columns['columns']
# unique_columns_ = list(set(columns_))
# df_train_add = pd.read_pickle(f'{file_folder}/distance-is-all-you-need_train.gzde', compression='gzip')
# df_test_add = pd.read_pickle(f'{file_folder}/distance-is-all-you-need_test.gzde', compression='gzip')
# df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')[unique_columns_+['index','type', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso']]
# df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')[unique_columns_+['index', 'type', 'group']]
# df_train = pd.merge(df_train, df_train_add.rename(columns={'id':'index'}).drop(columns=['scalar_coupling_constant']), on='index')
# df_test = pd.merge(df_test, df_test_add.rename(columns={'id':'index'}), on='index')
# df_train.to_pickle('tmp_df_train', compression='gzip')
# df_test.to_pickle('tmp_df_test', compression='gzip')


In [ ]:
"""
y=scalar_coupling_constant, type=0
556
{'columns': ['tertiary_distance_2', 'dist_H_2_y', 'dist_H_0_y', 'distC0', 'inv_dist1R', 'dist_C_2_x', 'dist_to_type_std', 'dist_C_2_y', 'dist_C_0_x', 'atom_1_n_bonds', 'd_5_1', 'yukawa_H.y', 'd_4_1', 'd_4_2', 'd_4_3', 'd_3_0', 'dist_H_2_x', 'd_4_0', 'tertiary_distance_4', 'tertiary_angle_1', 'd_3_1', 'dist_H_1_y', 'coulomb_H.y', 'dist_C_4_x', 'atom_index_farthest_0', 'distance_farthest_0', 'tertiary_angle_2', 'distance_c1', 'atom_1_bond_lengths_std', 'inv_dist0R', 'eem_0', 'inv_distPR', 'dist_N_0_y', 'dist_H_3_x', 'distC1', 'tertiary_atom_0', 'dist_H_1_x', 'dist_C_0_y', 'adC1', 'max_distance_y', 'tertiary_distance_5', 'dist_to_type_0_mean', 'atom_1_bond_lengths_mean', 'd_1_0', 'dist_O_1_y', 'tertiary_angle_0', 'yukawa_N.x', 'dist_C_4_y', 'atom_index_1_cycle_size_mean', 'tertiary_atom_1', 'inv_dist1', 'd_2_1', 'dist_C_1_y', 'adC2', 'inv_dist1E', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_1_mean', 'dist_to_type_mean', 'adC3', 'dist_O_0_y', 'eem_1', 'adN1', 'tertiary_distance_3', 'dist_N_0_x', 'molecule_atom_index_0_dist_max_div', 'dist_O_1_x', 'dist_C_3_y', 'tertiary_angle_3', 'cos_f0', 'd_3_2', 'dist_O_0_x', 'dist_H_3_y', 'dist_C_3_x', 'tertiary_angle_4'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 3, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=1
331
{'columns': ['yukawa_N.x', 'inv_distPR', 'molecule_atom_index_0_dist_mean_div', 'dist_C_0_y', 'dist_N_1_x', 'dist_C_1_x', 'd_4_1', 'molecule_atom_1_dist_min_diff', 'dist_N_0_y', 'cos_f1', 'dist_O_0_x', 'tertiary_distance_5', 'vander_N.x', 'dist_C_3_x', 'coulomb_H.x', 'dist_C_3_y', 'd_9_2', 'distance_center0', 'd_3_2', 'd_5_2', 'dist_C_2_x', 'dist_to_type_0_mean', 'dist_N_0_x', 'dist_O_0_y', 'd_4_3', 'dist_H_1_x', 'atom_1_bond_lengths_max', 'atom_1_bond_lengths_std', 'd_3_1', 'dist_H_3_y', 'tertiary_distance_1', 'molecule_atom_index_0_dist_max_div', 'inv_distP', 'tertiary_angle_3', 'tertiary_distance_0', 'linkN', 'd_2_0', 'd_4_2', 'd_5_3', 'dist_to_type_std', 'd_4_0', 'd_3_0', 'dist_H_0_y', 'dist_C_0_x', 'tertiary_angle_0', 'dist_N_1_y', 'adC3', 'inv_dist1R', 'dist_H_1_y', 'eem_1', 'dist_C_1_y', 'tertiary_distance_3', 'd_2_1', 'dist_H_2_y', 'tertiary_distance_4', 'link0', 'tertiary_angle_2', 'd_1_0', 'dist_C_2_y', 'tertiary_atom_0', 'yukawa_H.y', 'molecule_name.1', 'distance_c1', 'inv_dist0R', 'max_molecule_atom_0_dist_xyz', 'atom_1_bond_lengths_mean', 'cos_center0', 'cos_center0_center1', 'inv_dist1', 'eem_0', 'vander_H.x', 'tertiary_distance_2', 'dist_C_4_x', 'distN0', 'dist_H_0_x', 'max_distance_y', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 3, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=2
702
{'columns': ['molecule_atom_index_1_dist_min_diff', 'tertiary_angle_1', 'dist_C_0_y', 'tertiary_atom_2', 'd_6_2', 'dist_C_1_x', 'atom_8', 'd_4_1', 'd_9_1', 'dist_N_0_y', 'cos_c1', 'd_7_0', 'dist_O_0_x', 'tertiary_distance_5', 'dist_C_3_x', 'dist_O_1_x', 'dist_C_3_y', 'd_7_2', 'd_3_2', 'd_5_2', 'd_6_0', 'dist_C_2_x', 'dist_N_0_x', 'dist_to_type_0_mean', 'molecule_atom_index_0_dist_max_diff', 'dist_O_0_y', 'd_4_3', 'dist_H_1_x', 'atom_1_bond_lengths_std', 'd_3_1', 'd_5_0', 'tertiary_distance_1', 'molecule_atom_index_0_dist_max_div', 'adC1', 'tertiary_angle_3', 'atom_4', 'tertiary_distance_0', 'd_2_0', 'atom_3', 'cos_c0_c1', 'd_4_2', 'd_5_3', 'd_4_0', 'd_6_3', 'd_3_0', 'dist_H_0_y', 'dist_C_0_x', 'adN1', 'bond_atom', 'd_8_2', 'tertiary_angle_0', 'distC0', 'adC3', 'inv_dist1R', 'd_8_1', 'd_7_1', 'eem_1', 'dist_C_1_y', 'tertiary_distance_3', 'atom_index_1_cycle_size_mean', 'inv_dist0', 'd_2_1', 'dist_H_2_y', 'atom_1_n_bonds', 'tertiary_angle_2', 'd_1_0', 'molecule_atom_index_0_dist_mean_diff', 'dist_C_2_y', 'tertiary_atom_1', 'vander_O.y', 'tertiary_atom_0', 'distC1', 'adC2', 'atom_7', 'max_molecule_atom_0_dist_xyz', 'atom_1_bond_lengths_mean', 'dist_C_4_y', 'd_5_1', 'molecule_type_dist_max', 'd_6_1', 'distance_farthest_0', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_2', 'atom_5', 'dist_H_0_x', 'cos_c0', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 3, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=3
986
{'columns': ['molecule_atom_index_0_dist_min_diff', 'inv_distPR', 'tertiary_angle_1', 'dist_C_0_y', 'tertiary_atom_2', 'd_6_2', 'dist_C_1_x', 'd_4_1', 'dist_N_0_y', 'cos_c1', 'dist_to_type_mean', 'yukawa_H.x', 'cos_f1', 'dist_O_0_x', 'dist_H_3_x', 'dist_C_3_x', 'dist_C_3_y', 'd_7_2', 'dist_O_1_y', 'd_3_2', 'd_5_2', 'dist_C_2_x', 'inv_distPE', 'dist_N_0_x', 'dist_to_type_0_mean', 'molecule_dist_min', 'dist_O_0_y', 'd_4_3', 'dist_to_type_1_mean', 'dist_H_1_x', 'd_3_1', 'dist_H_3_y', 'd_5_0', 'tertiary_distance_1', 'molecule_atom_index_0_dist_max_div', 'inv_distP', 'tertiary_angle_3', 'adC4', 'd_2_0', 'atom_3', 'cos_c0_c1', 'd_4_2', 'd_5_3', 'dist_to_type_std', 'd_4_0', 'd_6_3', 'd_3_0', 'dist_H_0_y', 'dist_C_0_x', 'adN1', 'tertiary_angle_0', 'yukawa_C.x', 'mean_molecule_atom_0_dist_xyz', 'adC3', 'dist_H_1_y', 'dist_H_2_x', 'dist_C_1_y', 'tertiary_distance_3', 'coulomb_H.y', 'd_2_1', 'dist_H_2_y', 'vander_H.y', 'link0', 'tertiary_distance_4', 'vander_C.x', 'tertiary_angle_2', 'd_1_0', 'dist_C_2_y', 'tertiary_atom_1', 'yukawa_H.y', 'dist_H_4_y', 'mean_molecule_atom_1_dist_xyz', 'adC2', 'd_5_1', 'molecule_type_dist_max', 'vander_N.y', 'distance_farthest_0', 'molecule_atom_index_0_dist_min_div', 'vander_H.x', 'tertiary_distance_2', 'atom_5', 'cos_c0', 'vander_O.x', 'cos_f0_f1', 'dist_H_0_x', 'max_distance_y', 'dist_xyz', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 3, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=4
164
{'columns': ['dist_C_1_y', 'dist_C_1_x', 'atom_3', 'dist_C_0_y', 'd_4_2', 'dist_H_0_x', 'min_molecule_atom_0_dist_xyz', 'd_4_0', 'd_3_0', 'd_5_3', 'cos_c0_c1', 'adC3', 'tertiary_distance_1', 'cos_c0', 'tertiary_distance_3', 'inv_distPR', 'dist_O_0_y', 'molecule_atom_index_1_dist_min_diff', 'tertiary_atom_1', 'd_5_0', 'd_6_3', 'adC2', 'vander_C.x', 'd_3_2', 'd_6_0', 'dist_N_0_x', 'dist_C_3_y', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_std_div', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_0', 'inv_distP', 'd_4_3', 'dist_O_0_x', 'cos_f0', 'molecule_atom_index_0_dist_std_diff', 'yukawa_H.x', 'd_2_1', 'd_3_1', 'inv_dist0R', 'tertiary_distance_2', 'd_5_1', 'tertiary_atom_0', 'd_4_1', 'atom_1_bond_lengths_mean', 'inv_dist0', 'atom_1_bond_lengths_min'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 3, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=5
229
{'columns': ['dist_C_1_y', 'dist_C_1_x', 'atom_3', 'dist_C_0_y', 'd_4_2', 'dist_H_0_x', 'min_molecule_atom_0_dist_xyz', 'd_4_0', 'd_3_0', 'd_5_3', 'cos_c0_c1', 'adC3', 'tertiary_distance_1', 'cos_c0', 'tertiary_distance_3', 'inv_distPR', 'dist_O_0_y', 'molecule_atom_index_1_dist_min_diff', 'tertiary_atom_1', 'd_5_0', 'd_6_3', 'adC2', 'vander_C.x', 'd_3_2', 'd_6_0', 'dist_N_0_x', 'dist_C_3_y', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_std_div', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_0', 'inv_distP', 'd_4_3', 'dist_O_0_x', 'cos_f0', 'molecule_atom_index_0_dist_std_diff', 'yukawa_H.x', 'd_2_1', 'd_3_1', 'inv_dist0R', 'tertiary_distance_2', 'd_5_1', 'tertiary_atom_0', 'd_4_1', 'atom_1_bond_lengths_mean', 'inv_dist0', 'atom_1_bond_lengths_min'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 3, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=6
294
{'columns': ['dist_C_1_y', 'dist_C_1_x', 'atom_3', 'dist_C_0_y', 'd_4_2', 'dist_H_0_x', 'min_molecule_atom_0_dist_xyz', 'd_4_0', 'd_3_0', 'd_5_3', 'cos_c0_c1', 'adC3', 'tertiary_distance_1', 'cos_c0', 'tertiary_distance_3', 'inv_distPR', 'dist_O_0_y', 'molecule_atom_index_1_dist_min_diff', 'tertiary_atom_1', 'd_5_0', 'd_6_3', 'adC2', 'vander_C.x', 'd_3_2', 'd_6_0', 'dist_N_0_x', 'dist_C_3_y', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_std_div', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_0', 'inv_distP', 'd_4_3', 'dist_O_0_x', 'cos_f0', 'molecule_atom_index_0_dist_std_diff', 'yukawa_H.x', 'd_2_1', 'd_3_1', 'inv_dist0R', 'tertiary_distance_2', 'd_5_1', 'tertiary_atom_0', 'd_4_1', 'atom_1_bond_lengths_mean', 'inv_dist0', 'atom_1_bond_lengths_min'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 3, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=7
211
{'columns': ['dist_to_type_1_mean', 'cos_f0', 'dist_O_0_x', 'atom_1_bond_lengths_min', 'd_5_2', 'cos_c1', 'd_2_1', 'molecule_atom_index_0_dist_min_div', 'atom_index_1_cycle_size_mean', 'd_4_2', 'inv_distPR', 'dist_N_0_x', 'dist_O_0_y', 'adC1', 'adC2', 'tertiary_angle_2', 'dist_C_2_y', 'molecule_atom_index_0_dist_std_diff', 'dist_C_2_x', 'dist_C_3_y', 'd_4_3', 'eem_1', 'molecule_atom_index_1_dist_min_diff', 'atom_1_bond_lengths_mean', 'tertiary_angle_1', 'inv_dist0R', 'd_4_0', 'd_4_1', 'mean_molecule_atom_0_dist_xyz', 'd_2_0', 'd_3_1', 'dist_C_1_x', 'dist_H_0_y', 'dist_C_0_y', 'd_5_3', 'tertiary_distance_3', 'd_3_2', 'dist_H_1_x', 'tertiary_distance_1', 'atom_1_n_bonds', 'd_8_3', 'cos_c0', 'tertiary_distance_2', 'tertiary_angle_0', 'cos_c0_c1', 'molecule_atom_index_0_dist_min_diff', 'yukawa_H.y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 3, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}

"""



In [7]:
# columns_ = ['tertiary_distance_2', 'dist_H_2_y', 'dist_H_0_y', 'distC0', 'inv_dist1R', 'dist_C_2_x', 'dist_to_type_std', 'dist_C_2_y', 'dist_C_0_x', 'atom_1_n_bonds', 'd_5_1', 'yukawa_H.y', 'd_4_1', 'd_4_2', 'd_4_3', 'd_3_0', 'dist_H_2_x', 'd_4_0', 'tertiary_distance_4', 'tertiary_angle_1', 'd_3_1', 'dist_H_1_y', 'coulomb_H.y', 'dist_C_4_x', 'atom_index_farthest_0', 'distance_farthest_0', 'tertiary_angle_2', 'distance_c1', 'atom_1_bond_lengths_std', 'inv_dist0R', 'eem_0', 'inv_distPR', 'dist_N_0_y', 'dist_H_3_x', 'distC1', 'tertiary_atom_0', 'dist_H_1_x', 'dist_C_0_y', 'adC1', 'max_distance_y', 'tertiary_distance_5', 'dist_to_type_0_mean', 'atom_1_bond_lengths_mean', 'd_1_0', 'dist_O_1_y', 'tertiary_angle_0', 'yukawa_N.x', 'dist_C_4_y', 'atom_index_1_cycle_size_mean', 'tertiary_atom_1', 'inv_dist1', 'd_2_1', 'dist_C_1_y', 'adC2', 'inv_dist1E', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_1_mean', 'dist_to_type_mean', 'adC3', 'dist_O_0_y', 'eem_1', 'adN1', 'tertiary_distance_3', 'dist_N_0_x', 'molecule_atom_index_0_dist_max_div', 'dist_O_1_x', 'dist_C_3_y', 'tertiary_angle_3', 'cos_f0', 'd_3_2', 'dist_O_0_x', 'dist_H_3_y', 'dist_C_3_x', 'tertiary_angle_4']

# mytrial = []

# y = 'scalar_coupling_constant'
# t = 0

# print(f'y={y}, type={t}')
# df_train=pd.read_pickle('tmp_df_train', compression='gzip')
# #         df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')

# df_train['y'] = df_train[y]
# df_train = df_train[df_train['type']==t]

# X_data = df_train[columns_].values.astype('float32')
# y_data = df_train['scalar_coupling_constant'].values.astype('float32')

# X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=128)
# print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

# param = {
#     'columns': columns_,
#     'cv': {
#         'cls': 'KFold',
#         'init':{
#             'n_splits': 3,
#             'shuffle': True,
#             'random_state': 42,
#         },
#     },
#     'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}},
#     'model': {
#         'cls': 'lgb.LGBMRegressor',
#         'init': {
# #             'learning_rate': 0.2833769330240482,
# #             'feature_fraction': 0.8818248470204605,
# #             'bagging_fraction': 0.8205197060908092,
# #             'min_data_in_leaf': 202,
# #             'lambda_l1': 0.017039063121824582,
# #             'lambda_l2': 0.8318702431636841,
# #             'max_bin': 100,
# #             'num_leaves': 255,
# #             'random_state': 3895,
# #             'n_jobs': 16,

#             'boosting_type': 'gbdt',
#             'learning_rate': 0.2,
#             'num_leaves': 256,
#             'min_child_samples': 79,
#             'max_depth': 9,
#             'subsample_freq': 1,
#             'subsample': 0.9,
#             'bagging_seed': 11,
#             'reg_alpha': 0.1,
#             'reg_lambda': 0.3,
#             'colsample_bytree': 1.0
#         },
#         'fit': {}
#     },
#     'metric': 'mean_absolute_error'
# }

# LGB_PARAMS = param['model']['init']
# model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
# model.fit(X_train, y_train, 
#         eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae',
#         verbose=100, early_stopping_rounds=100)

# y_pred = model.predict(X_val)
# np.log(mean_absolute_error(y_val, y_pred)), mean_absolute_error(y_val, y_pred)

        
# # df_his, df_feature_importances, df_valid_pred, df_test_pred =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), param, f'modeling for {t}', df_test=None, trial=mytrial, is_output_feature_importance=False, trial_level=1)

In [8]:
mytrial = []

In [9]:
# , 'fc', 'sd','pso','dso'
for y in ['scalar_coupling_constant', 'fc', 'sd','pso','dso']:

    for t in np.arange(7,8,1):
        
        print(f'y={y}, type={t}')
        df_train=pd.read_pickle('tmp_df_train', compression='gzip')
#         df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
        
        df_train['y'] = df_train[y]
        df_train = df_train[df_train['type']==t]
        n_samples = 500000 if df_train.shape[0] > 500000 else df_train.shape[0]
        df_train = df_train.sample(n_samples).reset_index(drop=True)
#         df_test = df_test[df_test['type']==t].reset_index(drop=True)
        df_test = pd.DataFrame()
        

        param = {
            'columns': df_train.columns.drop(['index', 'y','group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso']).tolist(),
            'cv': {
                'cls': 'KFold',
                'init':{
                    'n_splits': 3,
                    'shuffle': True,
                    'random_state': 42,
                },
            },
            'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}},
            'model': {
                'cls': 'lgb.LGBMRegressor',
                'init': {
                    'learning_rate': 0.2833769330240482,
                    'feature_fraction': 0.8818248470204605,
                    'bagging_fraction': 0.8205197060908092,
                    'min_data_in_leaf': 202,
                    'lambda_l1': 0.017039063121824582,
                    'lambda_l2': 0.8318702431636841,
                    'max_bin': 100,
                    'num_leaves': 255,
                    'random_state': 3895,
#                     'n_estimators':1500,
                    'n_jobs': 16
                    
#                     'boosting_type': 'gbdt',
#                     'learning_rate': 0.2,
#                     'num_leaves': 256,
#                     'min_child_samples': 79,
#                     'max_depth': 9,
#                     'subsample_freq': 1,
#                     'subsample': 0.9,
#                     'bagging_seed': 11,
#                     'reg_alpha': 0.1,
#                     'reg_lambda': 0.3,
#                     'colsample_bytree': 1.0
                },
                'fit': {}
            },
            'metric': 'mean_absolute_error'
        }



        process_queue = PQueue(df_train, df_test, param, mytrial)
        sort_features = SortFeatureSelectTopNProcess(**{'top_n':200})
        select_topn = RFESelectTopNProcess(**{'n_features_remain':50, 'n_features_to_remove':10})
        remove_useless = RFERemoveUselessFeaturesProcess(**{})
        process_queue.insert_node(sort_features)
        process_queue.insert_node(select_topn)
        process_queue.insert_node(remove_useless)

        try:
            result = process_queue.run()
        except Exception as e:
            print(e.__str__())
        print(len(process_queue.trial))
        print(process_queue.param)
    

y=scalar_coupling_constant, type=7
211
{'columns': ['dist_to_type_1_mean', 'cos_f0', 'dist_O_0_x', 'atom_1_bond_lengths_min', 'd_5_2', 'cos_c1', 'd_2_1', 'molecule_atom_index_0_dist_min_div', 'atom_index_1_cycle_size_mean', 'd_4_2', 'inv_distPR', 'dist_N_0_x', 'dist_O_0_y', 'adC1', 'adC2', 'tertiary_angle_2', 'dist_C_2_y', 'molecule_atom_index_0_dist_std_diff', 'dist_C_2_x', 'dist_C_3_y', 'd_4_3', 'eem_1', 'molecule_atom_index_1_dist_min_diff', 'atom_1_bond_lengths_mean', 'tertiary_angle_1', 'inv_dist0R', 'd_4_0', 'd_4_1', 'mean_molecule_atom_0_dist_xyz', 'd_2_0', 'd_3_1', 'dist_C_1_x', 'dist_H_0_y', 'dist_C_0_y', 'd_5_3', 'tertiary_distance_3', 'd_3_2', 'dist_H_1_x', 'tertiary_distance_1', 'atom_1_n_bonds', 'd_8_3', 'cos_c0', 'tertiary_distance_2', 'tertiary_angle_0', 'cos_c0_c1', 'molecule_atom_index_0_dist_min_diff', 'yukawa_H.y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 3, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model'

KeyboardInterrupt: 

In [6]:
def objective(trial):
        
    learning_rate = trial.suggest_uniform('learning_rate', .01, .5)
    feature_fraction = trial.suggest_uniform('feature_fraction', .6, 1)
    bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.6, 1)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 200, 800)
    lambda_l1 = trial.suggest_loguniform('lambda_l1', 1e-6, 1e2)
    lambda_l2 = trial.suggest_loguniform('lambda_l2', 1e-6, 1e2)
    max_bin = trial.suggest_int('max_bin', 10, 100)
    num_leaves = trial.suggest_int('num_leaves', 4, 512)
        
    args = columns_list[t].copy()
    args['model']['init']={
                'learning_rate':learning_rate,
                'feature_fraction':feature_fraction,
                'bagging_fraction':bagging_fraction,
                'min_data_in_leaf':min_data_in_leaf,
                'lambda_l1':lambda_l1,
                'lambda_l2':lambda_l2,
                'max_bin':max_bin,
                'num_leaves':num_leaves,
                'n_jobs':16
    }
    
    df_his, df_feature_importances, df_valid_pred, df_test_pred =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), args, f'modeling for {t}', df_test=None, trial=mytrial, is_output_feature_importance=False, trial_level=1)
    val_metric_mean = np.mean(df_his.valid)
    return val_metric_mean



In [8]:
# df_trial = pd.DataFrame(mytrial)
# df_trial[df_trial['message']=='tune hyperparam'][['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff']].sort_values(by=['val_metric_mean']).head()

In [24]:
# df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
# df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
# df_train['y'] = df_train['scalar_coupling_constant']


df_train=pd.read_pickle(f'{file_folder}/tmp_df_train', compression='gzip')
df_test=pd.read_pickle(f'{file_folder}/tmp_df_test', compression='gzip')
df_train['y'] = df_train['scalar_coupling_constant']

In [25]:
train = pd.read_csv(f'{csv_file_folder}/train.csv')
train['molecule_index'] = train['molecule_name'].str.replace('dsgdb9nsd_', '').astype('int32')

df_add_structures = pd.read_pickle(f'{file_folder}/struct_eigen.pkl', compression='gzip') 
df_add_structures['molecule_index'] = df_add_structures['molecule_name'].str.replace('dsgdb9nsd_', '').astype('int32')
df_add_structures = df_add_structures.drop(columns=['molecule_name'])
def _map_atom_charges(df, structures, atom_idx):
    df = pd.merge(df, structures, how = 'left', left_on  = ['molecule_index', f'atom_index_{atom_idx}'], right_on = ['molecule_index',  'atom_index'])
    df = df.drop('atom_index', axis=1)
    rename_list = {}
    for col in structures.columns.drop(['molecule_index',  'atom_index']).tolist():
        rename_list[col] = f'atom_index_{atom_idx}_{col}'
    df = df.rename(columns=rename_list)
    return df

df_add = _map_atom_charges(train[['molecule_index', 'atom_index_0','atom_index_1']], df_add_structures, 0)
df_add = _map_atom_charges(df_add, df_add_structures, 1)
df_add = df_add.fillna(0)
df_add['index'] = df_add.index
df_add = df_add.drop(columns=['molecule_index', 'atom_index_0', 'atom_index_1'])

del train
del df_add_structures

In [26]:
df_train = pd.merge(df_train, df_add , how='left', on='index')
add_columns = df_add.columns.drop(['index']).tolist()
del df_add

In [13]:
# add_columns = df_add.columns.drop(['index']).tolist()

In [14]:
# oof_file_folder =  '../../data/oof'
# print([f for f in os.listdir(oof_file_folder) if f.startswith('lgbm')])
# df_oof_train = pd.DataFrame()
# df_oof_test = pd.DataFrame()
# for f in [f for f in os.listdir(oof_file_folder) if f.startswith('lgbm')]:
#     feat_name = f.split('_')[1]
#     if 'train' in f:
#         df_oof_i = pd.read_pickle(f'{oof_file_folder}/{f}')[['id', feat_name]].rename(columns={'id':'index', feat_name:f'oof_{feat_name}'})
#         df_oof_train = pd.concat([df_oof_train, df_oof_i], axis=1)
#     if 'test' in f:
#         df_oof_i = pd.read_pickle(f'{oof_file_folder}/{f}')[['id', feat_name]].rename(columns={'id':'index', feat_name:f'oof_{feat_name}'})
#         df_oof_test = pd.concat([df_oof_test, df_oof_i], axis=1)

# df_train = pd.concat([df_train, df_oof_train[['oof_sd','oof_fc','oof_pso','oof_dso']]], axis=1)
# df_test = pd.concat([df_test, df_oof_test[['oof_sd','oof_fc','oof_pso','oof_dso']]], axis=1)

In [27]:
columns_list = [
{'columns': add_columns + ['tertiary_distance_2', 'dist_H_2_y', 'dist_H_0_y', 'distC0', 'inv_dist1R', 'dist_C_2_x', 'dist_to_type_std', 'dist_C_2_y', 'dist_C_0_x', 'atom_1_n_bonds', 'd_5_1', 'yukawa_H.y', 'd_4_1', 'd_4_2', 'd_4_3', 'd_3_0', 'dist_H_2_x', 'd_4_0', 'tertiary_distance_4', 'tertiary_angle_1', 'd_3_1', 'dist_H_1_y', 'coulomb_H.y', 'dist_C_4_x', 'atom_index_farthest_0', 'distance_farthest_0', 'tertiary_angle_2', 'distance_c1', 'atom_1_bond_lengths_std', 'inv_dist0R', 'eem_0', 'inv_distPR', 'dist_N_0_y', 'dist_H_3_x', 'distC1', 'tertiary_atom_0', 'dist_H_1_x', 'dist_C_0_y', 'adC1', 'max_distance_y', 'tertiary_distance_5', 'dist_to_type_0_mean', 'atom_1_bond_lengths_mean', 'd_1_0', 'dist_O_1_y', 'tertiary_angle_0', 'yukawa_N.x', 'dist_C_4_y', 'atom_index_1_cycle_size_mean', 'tertiary_atom_1', 'inv_dist1', 'd_2_1', 'dist_C_1_y', 'adC2', 'inv_dist1E', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_1_mean', 'dist_to_type_mean', 'adC3', 'dist_O_0_y', 'eem_1', 'adN1', 'tertiary_distance_3', 'dist_N_0_x', 'molecule_atom_index_0_dist_max_div', 'dist_O_1_x', 'dist_C_3_y', 'tertiary_angle_3', 'cos_f0', 'd_3_2', 'dist_O_0_x', 'dist_H_3_y', 'dist_C_3_x', 'tertiary_angle_4'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': add_columns + ['yukawa_N.x', 'inv_distPR', 'molecule_atom_index_0_dist_mean_div', 'dist_C_0_y', 'dist_N_1_x', 'dist_C_1_x', 'd_4_1', 'molecule_atom_1_dist_min_diff', 'dist_N_0_y', 'cos_f1', 'dist_O_0_x', 'tertiary_distance_5', 'vander_N.x', 'dist_C_3_x', 'coulomb_H.x', 'dist_C_3_y', 'd_9_2', 'distance_center0', 'd_3_2', 'd_5_2', 'dist_C_2_x', 'dist_to_type_0_mean', 'dist_N_0_x', 'dist_O_0_y', 'd_4_3', 'dist_H_1_x', 'atom_1_bond_lengths_max', 'atom_1_bond_lengths_std', 'd_3_1', 'dist_H_3_y', 'tertiary_distance_1', 'molecule_atom_index_0_dist_max_div', 'inv_distP', 'tertiary_angle_3', 'tertiary_distance_0', 'linkN', 'd_2_0', 'd_4_2', 'd_5_3', 'dist_to_type_std', 'd_4_0', 'd_3_0', 'dist_H_0_y', 'dist_C_0_x', 'tertiary_angle_0', 'dist_N_1_y', 'adC3', 'inv_dist1R', 'dist_H_1_y', 'eem_1', 'dist_C_1_y', 'tertiary_distance_3', 'd_2_1', 'dist_H_2_y', 'tertiary_distance_4', 'link0', 'tertiary_angle_2', 'd_1_0', 'dist_C_2_y', 'tertiary_atom_0', 'yukawa_H.y', 'molecule_name.1', 'distance_c1', 'inv_dist0R', 'max_molecule_atom_0_dist_xyz', 'atom_1_bond_lengths_mean', 'cos_center0', 'cos_center0_center1', 'inv_dist1', 'eem_0', 'vander_H.x', 'tertiary_distance_2', 'dist_C_4_x', 'distN0', 'dist_H_0_x', 'max_distance_y', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': add_columns + ['molecule_atom_index_1_dist_min_diff', 'tertiary_angle_1', 'dist_C_0_y', 'tertiary_atom_2', 'd_6_2', 'dist_C_1_x', 'atom_8', 'd_4_1', 'd_9_1', 'dist_N_0_y', 'cos_c1', 'd_7_0', 'dist_O_0_x', 'tertiary_distance_5', 'dist_C_3_x', 'dist_O_1_x', 'dist_C_3_y', 'd_7_2', 'd_3_2', 'd_5_2', 'd_6_0', 'dist_C_2_x', 'dist_N_0_x', 'dist_to_type_0_mean', 'molecule_atom_index_0_dist_max_diff', 'dist_O_0_y', 'd_4_3', 'dist_H_1_x', 'atom_1_bond_lengths_std', 'd_3_1', 'd_5_0', 'tertiary_distance_1', 'molecule_atom_index_0_dist_max_div', 'adC1', 'tertiary_angle_3', 'atom_4', 'tertiary_distance_0', 'd_2_0', 'atom_3', 'cos_c0_c1', 'd_4_2', 'd_5_3', 'd_4_0', 'd_6_3', 'd_3_0', 'dist_H_0_y', 'dist_C_0_x', 'adN1', 'bond_atom', 'd_8_2', 'tertiary_angle_0', 'distC0', 'adC3', 'inv_dist1R', 'd_8_1', 'd_7_1', 'eem_1', 'dist_C_1_y', 'tertiary_distance_3', 'atom_index_1_cycle_size_mean', 'inv_dist0', 'd_2_1', 'dist_H_2_y', 'atom_1_n_bonds', 'tertiary_angle_2', 'd_1_0', 'molecule_atom_index_0_dist_mean_diff', 'dist_C_2_y', 'tertiary_atom_1', 'vander_O.y', 'tertiary_atom_0', 'distC1', 'adC2', 'atom_7', 'max_molecule_atom_0_dist_xyz', 'atom_1_bond_lengths_mean', 'dist_C_4_y', 'd_5_1', 'molecule_type_dist_max', 'd_6_1', 'distance_farthest_0', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_2', 'atom_5', 'dist_H_0_x', 'cos_c0', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': add_columns + ['molecule_atom_index_0_dist_min_diff', 'inv_distPR', 'tertiary_angle_1', 'dist_C_0_y', 'tertiary_atom_2', 'd_6_2', 'dist_C_1_x', 'd_4_1', 'dist_N_0_y', 'cos_c1', 'dist_to_type_mean', 'yukawa_H.x', 'cos_f1', 'dist_O_0_x', 'dist_H_3_x', 'dist_C_3_x', 'dist_C_3_y', 'd_7_2', 'dist_O_1_y', 'd_3_2', 'd_5_2', 'dist_C_2_x', 'inv_distPE', 'dist_N_0_x', 'dist_to_type_0_mean', 'molecule_dist_min', 'dist_O_0_y', 'd_4_3', 'dist_to_type_1_mean', 'dist_H_1_x', 'd_3_1', 'dist_H_3_y', 'd_5_0', 'tertiary_distance_1', 'molecule_atom_index_0_dist_max_div', 'inv_distP', 'tertiary_angle_3', 'adC4', 'd_2_0', 'atom_3', 'cos_c0_c1', 'd_4_2', 'd_5_3', 'dist_to_type_std', 'd_4_0', 'd_6_3', 'd_3_0', 'dist_H_0_y', 'dist_C_0_x', 'adN1', 'tertiary_angle_0', 'yukawa_C.x', 'mean_molecule_atom_0_dist_xyz', 'adC3', 'dist_H_1_y', 'dist_H_2_x', 'dist_C_1_y', 'tertiary_distance_3', 'coulomb_H.y', 'd_2_1', 'dist_H_2_y', 'vander_H.y', 'link0', 'tertiary_distance_4', 'vander_C.x', 'tertiary_angle_2', 'd_1_0', 'dist_C_2_y', 'tertiary_atom_1', 'yukawa_H.y', 'dist_H_4_y', 'mean_molecule_atom_1_dist_xyz', 'adC2', 'd_5_1', 'molecule_type_dist_max', 'vander_N.y', 'distance_farthest_0', 'molecule_atom_index_0_dist_min_div', 'vander_H.x', 'tertiary_distance_2', 'atom_5', 'cos_c0', 'vander_O.x', 'cos_f0_f1', 'dist_H_0_x', 'max_distance_y', 'dist_xyz', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': add_columns + ['dist_C_1_y', 'dist_C_1_x', 'atom_3', 'dist_C_0_y', 'd_4_2', 'dist_H_0_x', 'min_molecule_atom_0_dist_xyz', 'd_4_0', 'd_3_0', 'd_5_3', 'cos_c0_c1', 'adC3', 'tertiary_distance_1', 'cos_c0', 'tertiary_distance_3', 'inv_distPR', 'dist_O_0_y', 'molecule_atom_index_1_dist_min_diff', 'tertiary_atom_1', 'd_5_0', 'd_6_3', 'adC2', 'vander_C.x', 'd_3_2', 'd_6_0', 'dist_N_0_x', 'dist_C_3_y', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_std_div', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_0', 'inv_distP', 'd_4_3', 'dist_O_0_x', 'cos_f0', 'molecule_atom_index_0_dist_std_diff', 'yukawa_H.x', 'd_2_1', 'd_3_1', 'inv_dist0R', 'tertiary_distance_2', 'd_5_1', 'tertiary_atom_0', 'd_4_1', 'atom_1_bond_lengths_mean', 'inv_dist0', 'atom_1_bond_lengths_min'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': add_columns + ['dist_C_1_y', 'dist_C_1_x', 'atom_3', 'dist_C_0_y', 'd_4_2', 'dist_H_0_x', 'min_molecule_atom_0_dist_xyz', 'd_4_0', 'd_3_0', 'd_5_3', 'cos_c0_c1', 'adC3', 'tertiary_distance_1', 'cos_c0', 'tertiary_distance_3', 'inv_distPR', 'dist_O_0_y', 'molecule_atom_index_1_dist_min_diff', 'tertiary_atom_1', 'd_5_0', 'd_6_3', 'adC2', 'vander_C.x', 'd_3_2', 'd_6_0', 'dist_N_0_x', 'dist_C_3_y', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_std_div', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_0', 'inv_distP', 'd_4_3', 'dist_O_0_x', 'cos_f0', 'molecule_atom_index_0_dist_std_diff', 'yukawa_H.x', 'd_2_1', 'd_3_1', 'inv_dist0R', 'tertiary_distance_2', 'd_5_1', 'tertiary_atom_0', 'd_4_1', 'atom_1_bond_lengths_mean', 'inv_dist0', 'atom_1_bond_lengths_min'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': add_columns + ['dist_C_1_y', 'dist_C_1_x', 'atom_3', 'dist_C_0_y', 'd_4_2', 'dist_H_0_x', 'min_molecule_atom_0_dist_xyz', 'd_4_0', 'd_3_0', 'd_5_3', 'cos_c0_c1', 'adC3', 'tertiary_distance_1', 'cos_c0', 'tertiary_distance_3', 'inv_distPR', 'dist_O_0_y', 'molecule_atom_index_1_dist_min_diff', 'tertiary_atom_1', 'd_5_0', 'd_6_3', 'adC2', 'vander_C.x', 'd_3_2', 'd_6_0', 'dist_N_0_x', 'dist_C_3_y', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_std_div', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_0', 'inv_distP', 'd_4_3', 'dist_O_0_x', 'cos_f0', 'molecule_atom_index_0_dist_std_diff', 'yukawa_H.x', 'd_2_1', 'd_3_1', 'inv_dist0R', 'tertiary_distance_2', 'd_5_1', 'tertiary_atom_0', 'd_4_1', 'atom_1_bond_lengths_mean', 'inv_dist0', 'atom_1_bond_lengths_min'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': add_columns + ['dist_to_type_1_mean', 'cos_f0', 'dist_O_0_x', 'atom_1_bond_lengths_min', 'd_5_2', 'cos_c1', 'd_2_1', 'molecule_atom_index_0_dist_min_div', 'atom_index_1_cycle_size_mean', 'd_4_2', 'inv_distPR', 'dist_N_0_x', 'dist_O_0_y', 'adC1', 'adC2', 'tertiary_angle_2', 'dist_C_2_y', 'molecule_atom_index_0_dist_std_diff', 'dist_C_2_x', 'dist_C_3_y', 'd_4_3', 'eem_1', 'molecule_atom_index_1_dist_min_diff', 'atom_1_bond_lengths_mean', 'tertiary_angle_1', 'inv_dist0R', 'd_4_0', 'd_4_1', 'mean_molecule_atom_0_dist_xyz', 'd_2_0', 'd_3_1', 'dist_C_1_x', 'dist_H_0_y', 'dist_C_0_y', 'd_5_3', 'tertiary_distance_3', 'd_3_2', 'dist_H_1_x', 'tertiary_distance_1', 'atom_1_n_bonds', 'd_8_3', 'cos_c0', 'tertiary_distance_2', 'tertiary_angle_0', 'cos_c0_c1', 'molecule_atom_index_0_dist_min_diff', 'yukawa_H.y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_estimators':3000, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
]

In [15]:
# mytrial=[]
# df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
# for t in  df_train.type.unique().tolist()[-2:]:
#     study = optuna.create_study()
#     study.optimize(objective, n_trials=200)
#     print(study.best_params)

In [16]:
# param_model_init_list = [
# {'learning_rate': 0.4847973878400747, 'feature_fraction': 0.951715013033655, 'bagging_fraction': 0.6640125849570521, 'min_data_in_leaf': 437, 'lambda_l1': 0.0001680598586558378, 'lambda_l2': 5.156758691503555e-06, 'max_bin': 56, 'num_leaves': 490},
# {'learning_rate': 0.4041014002815698, 'feature_fraction': 0.9333037289627406, 'bagging_fraction': 0.7450389220980993, 'min_data_in_leaf': 243, 'lambda_l1': 3.7722915702800837e-06, 'lambda_l2': 4.243139074965437e-05, 'max_bin': 100, 'num_leaves': 506},
# {'learning_rate': 0.3864250628338235, 'feature_fraction': 0.934428347076253, 'bagging_fraction': 0.8030642265243596, 'min_data_in_leaf': 203, 'lambda_l1': 0.022599448912350148, 'lambda_l2': 4.0097557007881285e-05, 'max_bin': 63, 'num_leaves': 301},
# {'learning_rate': 0.3310621175605656, 'feature_fraction': 0.9096874372780241, 'bagging_fraction': 0.8461946729271688, 'min_data_in_leaf': 202, 'lambda_l1': 4.416715454062167e-05, 'lambda_l2': 97.7284456360912, 'max_bin': 73, 'num_leaves': 341},
# {'learning_rate': 0.4360511696328724, 'feature_fraction': 0.9267135240558227, 'bagging_fraction': 0.8834671928769631, 'min_data_in_leaf': 426, 'lambda_l1': 0.00740756930750945, 'lambda_l2': 3.042623853713187, 'max_bin': 98, 'num_leaves': 508},
# {'learning_rate': 0.4180174094678352, 'feature_fraction': 0.9626050915651176, 'bagging_fraction': 0.6880017140331517, 'min_data_in_leaf': 340, 'lambda_l1': 1.1359051360278207, 'lambda_l2': 0.0008619870627273151, 'max_bin': 82, 'num_leaves': 511},
# {'learning_rate': 0.4540923930379812, 'feature_fraction': 0.971839776738276, 'bagging_fraction': 0.6672418499150679, 'min_data_in_leaf': 534, 'lambda_l1': 0.08926785662993982, 'lambda_l2': 2.835575484786637e-05, 'max_bin': 92, 'num_leaves': 509},
# {'learning_rate': 0.3069006560814881, 'feature_fraction': 0.9963388008472429, 'bagging_fraction': 0.8793632084474166, 'min_data_in_leaf': 217, 'lambda_l1': 1.1982777608084696, 'lambda_l2': 26.943766780064788, 'max_bin': 80, 'num_leaves': 461}
# ]

# params_list=[]
# for param, model_init in zip(columns_list, param_model_init_list):
#     param['model']['init'] = model_init
#     params_list.append(param)

In [28]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist():
    df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), columns_list[t], f'modeling for {t}', df_test=None, trial=mytrial, is_output_feature_importance=False, trial_level=0)
    df_his = pd.concat([df_his, df_his_i], axis=0)
    df_feature_importances = pd.concat([df_feature_importances, df_feature_importances_i], axis=0)
    df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
    df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=0)
    

df_valid_pred = df_valid_pred.sort_values(by=['index']).reset_index(drop=True)
# df_test_pred = df_test_pred.sort_values(by=['index']).reset_index(drop=True)

KeyError: 'index'

In [29]:
df_trial = pd.DataFrame(mytrial)
df_trial['trn_val_metric_diff_rate'] = df_trial['trn_val_metric_diff'] / df_trial['train_metric_mean']
df_trial['log_val_mae'] = df_trial['val_metric_mean'].apply(lambda x : np.log(x))
print(mean_absolute_error(df_valid_pred.sort_values(by=['index']).reset_index(drop=True).predict.values, df_train.reset_index(drop=True).y.values))
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'trn_val_metric_diff_rate', 'message', 'log_val_mae']]

0.30716498427613625


,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff,trn_val_metric_diff_rate,message,log_val_mae
0,2019-08-17 18:08:40.348853,modeling for 0,98,0.075624,0.659857,0.584233,7.725526,modeling for 0,-0.415733
1,2019-08-17 19:07:44.749280,modeling for 3,112,0.005203,0.158181,0.152978,29.401667,modeling for 3,-1.844015
2,2019-08-17 19:26:09.252896,modeling for 1,101,0.004615,0.387191,0.382576,82.898392,modeling for 1,-0.948837
3,2019-08-17 20:04:03.334713,modeling for 4,72,0.000558,0.174294,0.173736,311.118556,modeling for 4,-1.747011
4,2019-08-17 21:37:43.703132,modeling for 2,111,0.052656,0.257090,0.204434,3.882455,modeling for 2,-1.358331
5,2019-08-17 22:35:00.043051,modeling for 6,72,0.015936,0.179346,0.163409,10.253807,modeling for 6,-1.718439
6,2019-08-18 00:13:20.176635,modeling for 5,72,0.089534,0.294850,0.205315,2.293152,modeling for 5,-1.221290
7,2019-08-18 00:59:45.200085,modeling for 7,71,0.000801,0.125110,0.124309,155.107509,modeling for 7,-2.078561


In [31]:
df_trial = pd.DataFrame(mytrial)
df_trial['trn_val_metric_diff_rate'] = df_trial['trn_val_metric_diff'] / df_trial['train_metric_mean']
df_trial['log_val_mae'] = df_trial['val_metric_mean'].apply(lambda x : np.log(x))
print(mean_absolute_error(df_valid_pred.sort_values(by=['index']).reset_index(drop=True).predict.values, df_train.reset_index(drop=True).y.values))
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'trn_val_metric_diff_rate', 'message', 'log_val_mae']]

0.283506349135093


,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff,trn_val_metric_diff_rate,message,log_val_mae
0,2019-08-15 10:39:59.213047,modeling for 0,74,0.119730,0.620052,0.500322,4.178773,modeling for 0,-0.477952
1,2019-08-15 13:42:36.123079,modeling for 3,88,0.010164,0.148630,0.138466,13.622600,modeling for 3,-1.906292
2,2019-08-15 13:58:27.208197,modeling for 1,77,0.005992,0.356884,0.350892,58.557778,modeling for 1,-1.030344
3,2019-08-15 14:44:40.291754,modeling for 4,48,0.005803,0.146795,0.140992,24.295292,modeling for 4,-1.918717
4,2019-08-15 18:21:57.496771,modeling for 2,87,0.069108,0.236843,0.167735,2.427154,modeling for 2,-1.440359
5,2019-08-15 19:45:46.663349,modeling for 6,48,0.032048,0.157919,0.125871,3.927608,modeling for 6,-1.845675
6,2019-08-15 22:09:19.347532,modeling for 5,48,0.113392,0.271739,0.158347,1.396451,modeling for 5,-1.302914
7,2019-08-15 23:12:30.828864,modeling for 7,47,0.004238,0.106440,0.102202,24.114797,modeling for 7,-2.240173


In [26]:
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'trn_val_metric_diff_rate', 'message', 'log_val_mae']]

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff,trn_val_metric_diff_rate,message,log_val_mae
0,2019-08-15 02:05:41.126228,modeling for 0,74,0.210362,0.666421,0.456060,2.167979,modeling for 0,-0.405833
1,2019-08-15 02:51:57.952391,modeling for 3,88,0.023252,0.157248,0.133996,5.762899,modeling for 3,-1.849933
2,2019-08-15 02:58:18.893885,modeling for 1,77,0.018283,0.381008,0.362726,19.839749,modeling for 1,-0.964934
3,2019-08-15 03:12:43.772104,modeling for 4,48,0.013993,0.157495,0.143502,10.255205,modeling for 4,-1.848363
4,2019-08-15 04:38:06.906174,modeling for 2,87,0.109666,0.258878,0.149211,1.360594,modeling for 2,-1.351400
5,2019-08-15 05:07:21.139056,modeling for 6,48,0.055038,0.170250,0.115212,2.093308,modeling for 6,-1.770487
6,2019-08-15 06:04:46.187347,modeling for 5,48,0.164317,0.297397,0.133080,0.809900,modeling for 5,-1.212686
7,2019-08-15 06:19:11.274392,modeling for 7,47,0.009743,0.114216,0.104473,10.723145,modeling for 7,-2.169668


In [38]:
idx=0.34
# df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['scalar_coupling_constant'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)*y_std+y_mean
df_submit['id'] = df_test_pred['index']
df_submit.to_csv('../../data/submission/submission_lgbm_{}.csv'.format(idx), index=False)